# Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/ISIR/RedBallGame/final

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/ISIR/RedBallGame/final


In [ ]:
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from collections import defaultdict
import torch
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer
from dataclasses import dataclass

In [ ]:
from utils.data_retrieval import process_annotations
from utils.categorisation import categorize_facial_mot, categorize_utterance_mot, categorize_gaze_mot, categorize_gaze_chi
from utils.multimodal_dataset import MultimodalDataset
from utils.dataset_split import DatasetSplit
from utils.train_loop import train_loop
from utils.evaluation_loop import evaluation_loop

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Data Preparation

In [ ]:
file_name = "A_12_20130925_1212_01_2.txt"

In [ ]:
process_annotations(file_name, "annotations_per_t.csv")

Per-second annotations saved to /content/drive/MyDrive/ISIR/RedBallGame/final/annotations_per_t.csv


## Annotations categorization

In [ ]:
df = pd.read_csv("annotations_per_t.csv")

In [ ]:
df['Annotation'] = df.apply(
    lambda row: categorize_facial_mot(row['Annotation']) if row['Tier'] == 'Facial@MOT' else
                categorize_utterance_mot(row['Annotation']) if row['Tier'] == 'Utterance@MOT' else
                categorize_gaze_mot(row['Annotation']) if row['Tier'] == 'Gaze@MOT' else
                categorize_gaze_chi(row['Annotation']) if row['Tier'] == 'Gaze@CHI' else
                row['Annotation'],
    axis=1
)

## Create objects

### Standard

In [ ]:
tiers = [
    "Action@MOT",
    "Gaze@MOT",
    "Utterance@MOT",
    "Prosody@MOT",
    "Facial@MOT",
    "Laughter@MOT",
    "Laughter@CHI",
    "Gaze@CHI"
]

pivot_df = df.pivot_table(
    index='Time',
    columns='Tier',
    values='Annotation',
    aggfunc=lambda x: x.iloc[0]
)

pivot_df = pivot_df[tiers]

pivot_df = pivot_df.where(pd.notnull(pivot_df), None)

records = pivot_df.to_dict(orient='records')

X = records.copy()

### With GazeRelation

In [ ]:
tiersGR = [
    "Action@MOT",
    "Gaze@MOT",
    "Utterance@MOT",
    "Prosody@MOT",
    "Facial@MOT",
    "Laughter@MOT",
    "Laughter@CHI",
    "Gaze@CHI",
    "GazeRelation"
]
pivot_gr_df = df.pivot_table(
    index='Time',
    columns='Tier',
    values='Annotation',
    aggfunc=lambda x: x.iloc[0]
)

records_gr = pivot_gr_df.to_dict(orient='records')

for record in records_gr:
    mot_target = record.get("Gaze@MOT")
    chi_target = record.get("Gaze@CHI")

    if chi_target == "mom" and mot_target == "child":
        label = "MA"  # MutualAttention
    elif mot_target == "invisible":
        label = "<UNK>"  # Unknown
    elif chi_target == "mom":
        label = "SA(CHI)"  # SingleAttention(CHI)
    elif mot_target == "child":
        label = "SA(MOT)"  # SingleAttention(MOT)
    elif chi_target == mot_target and chi_target is not None:
        label = "ShA"  # SharedAttention
    elif chi_target != mot_target:
        label = "L"  # LostAttention
    else:
        label = "<UNK>"  # Unknown

    record["GazeRelation"] = label

pivot_gr_df = pd.DataFrame(records_gr)

pivot_gr_df = pivot_gr_df[tiersGR]

pivot_gr_df = pivot_gr_df.where(pd.notnull(pivot_gr_df), None)

records_gr = pivot_gr_df.to_dict(orient='records')

X_gr = records_gr.copy()

## Retrieve vocabs

In [ ]:
values = defaultdict(set)

for r in records_gr:
    for key in r.keys():
        value = r[key]
        if value is None or value == '' or value == 'invisible':
            value = "<UNK>"  # use <UNK> token for missing
        values[key].add(value)

vocab_action    = {token: idx for idx, token in enumerate(sorted(values['Action@MOT']))}
vocab_gaze      = {token: idx for idx, token in enumerate(sorted(values['Gaze@MOT']))}
vocab_utterance = {token: idx for idx, token in enumerate(sorted(values['Utterance@MOT']))}
vocab_prosody   = {token: idx for idx, token in enumerate(sorted(values['Prosody@MOT']))}
vocab_facial    = {token: idx for idx, token in enumerate(sorted(values['Facial@MOT']))}
vocab_laughter  = {token: idx for idx, token in enumerate(sorted(values['Laughter@MOT']))}
vocab_laughter_chi = {token: idx for idx, token in enumerate(sorted(values['Laughter@CHI']))}
vocab_gaze_chi  = {token: idx for idx, token in enumerate(sorted(values['Gaze@CHI']))}
vocab_gazerelation  = {token: idx for idx, token in enumerate(sorted(values['GazeRelation']))}

In [ ]:
vocabs = {"action": vocab_action, "gaze": vocab_gaze, "utterance": vocab_utterance,
          "prosody": vocab_prosody, "facial": vocab_facial, "laughter": vocab_laughter,
          "laughter_chi": vocab_laughter_chi, "gaze_chi": vocab_gaze_chi}

vocabs_gr = {"action": vocab_action, "gaze": vocab_gaze, "utterance": vocab_utterance,
             "prosody": vocab_prosody, "facial": vocab_facial, "laughter": vocab_laughter,
             "gazerelation": vocab_gazerelation, "laughter_chi": vocab_laughter_chi, "gaze_chi": vocab_gaze_chi}

# Helper functions

In [ ]:
def get_data(isGazeRelation):
    if isGazeRelation:
        return X_gr, vocabs_gr
    else:
        return X, vocabs

## Data Loader (batches divsion)

In [ ]:
from torch.utils.data import DataLoader

# Finding the index of a value in a vocabulary
def lookup(vocab, value):
    if value is None:
        value = "<UNK>"
    return vocab.get(value, vocab["<UNK>"])

def collate_fn(batch):
    """
    Collate function for DataLoader without GazeRelation target.
    Processes a batch of data into input sequences and target tensors.
    """
    sequences, targets_list = zip(*batch)  # Separate inputs and targets

    target_laughter_indices = []
    target_gaze_indices = []

    # Convert each target value to an index
    for target_dict in targets_list:
        laughter_val = target_dict.get('Laughter@CHI')
        gaze_val = target_dict.get('Gaze@CHI')

        target_laughter_indices.append(lookup(vocab_laughter_chi, laughter_val))
        target_gaze_indices.append(lookup(vocab_gaze_chi, gaze_val))

    # Convert target index lists to tensors
    targets_laughter = torch.tensor(target_laughter_indices, dtype=torch.long)
    targets_gaze = torch.tensor(target_gaze_indices, dtype=torch.long)

    # Wrap each sequence dict in a list to create a sequence of length 1
    X_batch_sequences = [[seq_dict] for seq_dict in sequences]

    return X_batch_sequences, (targets_gaze, targets_laughter)

def collate_fn_gr(batch):
    """
    Collate function for DataLoader with GazeRelation target.
    Same as collate_fn but also processes the 'GazeRelation' target.
    """
    sequences, targets_list = zip(*batch)

    target_laughter_indices = []
    target_gaze_indices = []
    target_gazerelation_indices = []

    for target_dict in targets_list:
        laughter_val = target_dict.get('Laughter@CHI')
        gaze_val = target_dict.get('Gaze@CHI')
        gazerelation_val = target_dict.get('GazeRelation')

        target_laughter_indices.append(lookup(vocab_laughter_chi, laughter_val))
        target_gaze_indices.append(lookup(vocab_gaze_chi, gaze_val))
        target_gazerelation_indices.append(lookup(vocab_gazerelation, gazerelation_val))

    # Convert lists to tensors
    targets_laughter = torch.tensor(target_laughter_indices, dtype=torch.long)
    targets_gaze = torch.tensor(target_gaze_indices, dtype=torch.long)
    targets_gazerelation = torch.tensor(target_gazerelation_indices, dtype=torch.long)

    # Wrap each sequence dict in a list to create a sequence of length 1
    X_batch_sequences = [[seq_dict] for seq_dict in sequences]

    return X_batch_sequences, (targets_gaze, targets_laughter, targets_gazerelation)

def get_loader(X, y, batch_size, isGazeRelation):
    """
    Creates a DataLoader for the given dataset.
    """
    dataset = MultimodalDataset(X, y)
    if isGazeRelation:
        loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn_gr)
    else:
        loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
    return loader


In [ ]:
def get_loaders(X, isGazeRelation, isMixedPhases, train_size, val_size, n_parts):
    """
    Splits the dataset into train/validation/test sets and returns DataLoaders for each.
    """
    # If phases are not mixed, process the dataset as a single part
    if not isMixedPhases:
        n_parts = 1

    # Split the dataset into train, validation, and test subsets
    X_train, y_train, X_val, y_val, X_test, y_test = DatasetSplit(
        dataset=X,
        n_parts=n_parts,
        train_size=train_size,
        val_size=val_size
    ).main()

    # Determine batch sizes so that each phase is represented in a batch
    train_batch_size = len(X_train) // n_parts
    val_batch_size = len(X_val) // n_parts
    test_batch_size = len(X_test) // n_parts

    # Create DataLoaders for each subset
    train_loader = get_loader(X_train, y_train, batch_size=train_batch_size, isGazeRelation=isGazeRelation)
    val_loader = get_loader(X_val, y_val, batch_size=val_batch_size, isGazeRelation=isGazeRelation)
    test_loader = get_loader(X_test, y_test, batch_size=test_batch_size, isGazeRelation=isGazeRelation)

    return train_loader, val_loader, test_loader


## Training Model

In [ ]:
@dataclass
class DatasetParams:
    isGazeRelation: bool
    isMixedPhases: bool
    train_size: int
    val_size: int
    n_parts: int

@dataclass
class ModelParams:
    hidden_dim: int
    num_layers: int
    dropout: float
    input_size: int
    bidirectional: bool

@dataclass
class TrainParams:
    lr: float
    weight_decay: float
    num_epochs: int

In [ ]:
def training_model(model, dataset_params: DatasetParams, model_params: ModelParams, train_params: TrainParams):
    """
    Trains, validates, and evaluates the given model.
    """
    # Load dataset and corresponding vocabularies
    dataset, vocabs = get_data(dataset_params.isGazeRelation)

    model = model(
        vocabs,
        dataset_params.isGazeRelation,
        hidden_dim=model_params.hidden_dim,
        num_layers=model_params.num_layers,
        dropout=model_params.dropout,
        bidirectional=model_params.bidirectional,
        input_size=model_params.input_size
    ).to(device)

    train_loader, val_loader, test_loader = get_loaders(
        dataset,
        dataset_params.isGazeRelation,
        dataset_params.isMixedPhases,
        dataset_params.train_size,
        dataset_params.val_size,
        dataset_params.n_parts
    )

    optimizer = optim.Adam(model.parameters(), lr=train_params.lr, weight_decay=train_params.weight_decay)
    criterion = nn.CrossEntropyLoss()

    num_epochs = train_params.num_epochs

    print("Starting training...")
    for epoch in tqdm(range(num_epochs), desc="Training Epochs"):
        # ---------------- TRAINING ----------------
        model.train()
        model = train_loop(model, train_loader, criterion, optimizer, dataset_params.isGazeRelation, device)

        model.eval()
        with torch.no_grad():
            avg_train_loss, loss_gaze, loss_laughter, train_gaze_accuracy, train_gaze_precision, train_gaze_f1, train_laughter_accuracy, train_laughter_precision, train_laughter_f1 = evaluation_loop(
                model, train_loader, criterion, dataset_params.isGazeRelation, device
            )

        # ---------------- VALIDATION ----------------
        model.eval()
        with torch.no_grad():
            avg_val_loss, loss_gaze, loss_laughter, val_gaze_accuracy, val_gaze_precision, val_gaze_f1, val_laughter_accuracy, val_laughter_precision, val_laughter_f1 = evaluation_loop(
                model, val_loader, criterion, dataset_params.isGazeRelation, device
            )

        print(
            f"Epoch [{epoch+1}/{num_epochs}], "
            f"\tTrain Loss: {avg_train_loss:.4f}, Train Gaze Acc: {train_gaze_accuracy:.4f}, Train Gaze Prec: {train_gaze_precision:.4f}, Train Gaze F1: {train_gaze_f1:.4f}, Train Laughter Acc: {train_laughter_accuracy:.4f}, Train Laughter Prec: {train_laughter_precision:.4f}, Train Laughter F1: {train_laughter_f1:.4f}, "
            f"\n\tVal Loss: {avg_val_loss:.4f}, Val Gaze Acc: {val_gaze_accuracy:.4f}, Val Gaze Prec: {val_gaze_precision:.4f}, Val Gaze F1: {val_gaze_f1:.4f}, Val Laughter Acc: {val_laughter_accuracy:.4f}, Val Laughter Prec: {val_laughter_precision:.4f}, Val Laughter F1: {val_laughter_f1:.4f}"
        )

    print("Training finished.")

    # ---------------- TEST EVALUATION ----------------
    model.eval()
    with torch.no_grad():
        avg_test_loss, loss_gaze, loss_laughter, test_accuracy_gaze, test_gaze_precision, test_gaze_f1, test_accuracy_laughter, test_laughter_precision, test_laughter_f1 = evaluation_loop(
            model, test_loader, criterion, dataset_params.isGazeRelation, device
        )

    print("\nTest Set Evaluation:")
    print(
        f"Test Loss: {avg_test_loss:.4f}, Test Gaze Acc: {test_accuracy_gaze:.4f}, Test Gaze Prec: {test_gaze_precision:.4f}, Test Gaze F1: {test_gaze_f1:.4f}, "
        f"Test Laughter Acc: {test_accuracy_laughter:.4f}, Test Laughter Prec: {test_laughter_precision:.4f}, Test Laughter F1: {test_laughter_f1:.4f}"
    )

    return model


## Grid Search

In [ ]:
from sklearn.model_selection import ParameterGrid

def grid_search(model, dataset_params: DatasetParams, param_grid: dict):

    best_val_loss = float('inf')
    best_params = None
    results = []

    # Load dataset and vocabularies
    dataset, vocabs = get_data(dataset_params.isGazeRelation)

    # Convert param_grid dict into a list of all combinations
    param_grid = ParameterGrid(param_grid)

    print(f"Starting Grid Search with {len(param_grid)} combinations...")

    # Iterate over all hyperparameter combinations
    for i, params in enumerate(tqdm(param_grid, desc="Processing params", unit="set")):
        print(f"\n--- Running combination {i+1}/{len(param_grid)} ---")
        print(f"Parameters: {params}")

        # Create ModelParams and TrainParams dataclasses for the current combination
        model_params = ModelParams(
            hidden_dim=params['hidden_dim'],
            num_layers=params['num_layers'],
            dropout=params['dropout'],
            bidirectional=params['bidirectional'],
            input_size=416
        )
        train_params = TrainParams(
            lr=params['lr'],
            weight_decay=params['weight_decay'],
            num_epochs=params['num_epochs']
        )

        model_instance = model(
            vocabs,
            dataset_params.isGazeRelation,
            hidden_dim=model_params.hidden_dim,
            num_layers=model_params.num_layers,
            dropout=model_params.dropout,
            bidirectional=model_params.bidirectional,
            input_size=model_params.input_size
        ).to(device)

        train_loader, val_loader, test_loader = get_loaders(
            dataset,
            dataset_params.isGazeRelation,
            dataset_params.isMixedPhases,
            dataset_params.train_size,
            dataset_params.val_size,
            dataset_params.n_parts
        )

        optimizer = optim.Adam(model_instance.parameters(), lr=train_params.lr, weight_decay=train_params.weight_decay)
        criterion = nn.CrossEntropyLoss()

        # ---------------- TRAINING LOOP ----------------
        for epoch in tqdm(range(train_params.num_epochs), desc=f"Training Epochs (Params {params})"):
            model_instance.train()

            model_instance = train_loop(model_instance, train_loader, criterion, optimizer, dataset_params.isGazeRelation, device)

            model_instance.eval()
            with torch.no_grad():
                avg_val_loss, _, _, val_gaze_accuracy, val_gaze_precision, val_gaze_f1, val_laughter_accuracy, val_laughter_precision, val_laughter_f1 = evaluation_loop(
                    model_instance, val_loader, criterion, dataset_params.isGazeRelation, device
                )

        # ---------------- VALIDATION EVALUATION ----------------
        # Evaluate model on test set after training
        model_instance.eval()
        with torch.no_grad():
            avg_val_loss, _, _, val_gaze_accuracy, val_gaze_precision, val_gaze_f1, val_laughter_accuracy, val_laughter_precision, val_laughter_f1 = evaluation_loop(
                model_instance, test_loader, criterion, dataset_params.isGazeRelation, device
            )

        print(f"Validation Loss for this combo: {avg_val_loss:.4f}")

        results.append({
            'params': params,
            'val_loss': avg_val_loss,
            'val_gaze_accuracy': val_gaze_accuracy,
            'val_gaze_precision': val_gaze_precision,
            'val_gaze_f1': val_gaze_f1,
            'val_laughter_accuracy': val_laughter_accuracy,
            'val_laughter_precision': val_laughter_precision,
            'val_laughter_f1': val_laughter_f1,
        })

        # Update best parameters if this combination has lowest validation loss
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_params = params

    print("\n--- Grid Search Finished ---")
    print(f"Best parameters found: {best_params}")
    print(f"Best validation loss: {best_val_loss:.4f}")

    return results, best_params

# Single LSTM

## Model

In [ ]:
class EnfantReactionPredictor(nn.Module):
    """
      Neural network to predict child reactions (gaze, laughter, optionally gaze relation)

      Args:
          vocabs: dictionary of vocabularies for all modalities
          isGazeRelation: boolean - if True, use GazeRelation in addition to gaze/laughter
          hidden_dim
          num_layers
          dropout
          input_size
          bidirectional: if True, use bidirectional LSTM
    """
    def __init__(self,
                 vocabs,
                 isGazeRelation,
                 hidden_dim=16,
                 num_layers=2,
                 dropout=0.0,
                 input_size=40,
                 bidirectional=False):

        super().__init__()

        self.isGazeRelation = isGazeRelation

        # ---------------- VOCABS ----------------
        self.vocab_gaze = vocabs["gaze"]
        self.vocab_utterance = vocabs["utterance"]
        self.vocab_prosody = vocabs["prosody"]
        self.vocab_facial = vocabs["facial"]
        self.vocab_laughter = vocabs["laughter"]
        self.vocab_gaze_chi = vocabs["gaze_chi"]
        self.vocab_laughter_chi = vocabs["laughter_chi"]

        if self.isGazeRelation:
            self.vocab_gazerelation = vocabs["gazerelation"]

        # ---------------- EMBEDDINGS ----------------
        # Embedding layers for mother features
        self.embed_gaze = nn.Embedding(len(vocabs["gaze"]), 8)
        self.embed_utterance = nn.Embedding(len(vocabs["utterance"]), 8)
        self.embed_prosody = nn.Embedding(len(vocabs["prosody"]), 4)
        self.embed_facial = nn.Embedding(len(vocabs["facial"]), 8)
        self.embed_laughter = nn.Embedding(len(vocabs["laughter"]), 4)

        # Sentence encoder for action text using a pre-trained transformer
        self.sentence_encoder = SentenceTransformer('all-MiniLM-L6-v2')

        # Embeddings for child targets
        self.embed_laughter_chi = nn.Embedding(len(vocabs["laughter_chi"]), 4)
        self.embed_gaze_chi = nn.Embedding(len(vocabs["gaze_chi"]), 8)

        if self.isGazeRelation:
            self.embed_gazerelation = nn.Embedding(len(self.vocab_gazerelation), 4)

        # ---------------- LSTM ----------------
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,
            bidirectional=bidirectional
        )

        self.dropout = nn.Dropout(dropout)

        # Determine input size for decoder based on bidirectional LSTM
        decoder_input_dim = hidden_dim * (2 if bidirectional else 1)

        # ---------------- DECODERS ----------------
        # Linear layers to predict child gaze and laughter
        self.decoder_gaze = nn.Linear(decoder_input_dim, len(vocabs["gaze_chi"]))
        self.decoder_laughter = nn.Linear(decoder_input_dim, len(vocabs["laughter_chi"]))

        # Decoder for GazeRelation
        if self.isGazeRelation:
            self.decoder_gazerelation = nn.Linear(decoder_input_dim, len(self.vocab_gazerelation))

    # ---------------- LOOKUP ----------------
    def lookup(self, vocab, value):
        if value is None:
            value = "<UNK>"
        return vocab.get(value, vocab["<UNK>"])

    # ---------------- ENCODE INPUTS ----------------
    def encode_input_batch(self, batch_sequence_of_dicts, device):
        """
        Convert a batch of sequences of dictionaries into embeddings concatenated as LSTM input.

        """
        all_seqs = []

        for sequence in batch_sequence_of_dicts:
            seq = []
            for step_dict in sequence:
                # Convert mother features to indices
                gaze = torch.tensor([self.lookup(self.vocab_gaze, step_dict.get("Gaze@MOT"))], dtype=torch.long)
                utterance = torch.tensor([self.lookup(self.vocab_utterance, step_dict.get("Utterance@MOT"))], dtype=torch.long)
                prosody = torch.tensor([self.lookup(self.vocab_prosody, step_dict.get("Prosody@MOT"))], dtype=torch.long)
                facial = torch.tensor([self.lookup(self.vocab_facial, step_dict.get("Facial@MOT"))], dtype=torch.long)
                laughter = torch.tensor([self.lookup(self.vocab_laughter, step_dict.get("Laughter@MOT"))], dtype=torch.long)

                # Encode action text using sentence transformer
                action_text = step_dict.get("Action@MOT") or "<UNK>"
                encoded_action = torch.tensor(self.sentence_encoder.encode(action_text), dtype=torch.float32).unsqueeze(0)

                # Concatenate embeddings and action encoding
                step_emb = torch.cat([
                    self.embed_gaze(gaze),
                    self.embed_utterance(utterance),
                    self.embed_prosody(prosody),
                    self.embed_facial(facial),
                    self.embed_laughter(laughter),
                    encoded_action
                ], dim=-1)

                seq.append(step_emb)

            seq = torch.cat(seq, dim=0)
            all_seqs.append(seq)

        batch_tensor = torch.stack(all_seqs).to(device)
        return batch_tensor

    # ---------------- FORWARD PASS ----------------
    def forward(self, batch_sequence_of_dicts, device):

        # Encode batch sequences
        encoded_steps = self.encode_input_batch(batch_sequence_of_dicts, device)

        # Run through LSTM
        output, (h_n, c_n) = self.lstm(encoded_steps)

        # Apply decoders for each target
        logits_gaze = self.decoder_gaze(output.squeeze(1))
        logits_laughter = self.decoder_laughter(output.squeeze(1))

        if self.isGazeRelation:
            logits_gazerelation = self.decoder_gazerelation(output.squeeze(1))
            return logits_gaze, logits_laughter, logits_gazerelation

        return logits_gaze, logits_laughter


## Grid Search Results

### Results - Standard

In [ ]:
# Define the parameter grid
param_grid = {
    'hidden_dim': [128, 256],
    'num_layers': [1, 2],
    'dropout': [0.1, 0.3],
    'bidirectional': [False, True],
    'lr': [0.003],
    'weight_decay': [1e-5],
    'num_epochs': [10]
}



In [ ]:
# Define dataset parameters
dataset_params = DatasetParams(
    isGazeRelation=False,
    isMixedPhases=False,
    train_size=0.5,
    val_size=0.25,
    n_parts=1
)

# Run the grid search
results, best_params = grid_search(
    model=EnfantReactionPredictor,
    dataset_params=dataset_params,
    param_grid=param_grid
)

print("\nGrid Search Results:")
for res in results:
      print(res)

results_df = pd.DataFrame(results)

print("\nGrid Search Results DataFrame:")
display(results_df)

results_df.to_csv('grid_search_lstm_results_df.csv', index=False)

Starting Grid Search with 16 combinations...


Processing params:   0%|          | 0/16 [00:00<?, ?set/s]


--- Running combination 1/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(


Training Epochs (Params {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs':…

...

Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 2.3499

--- Running combination 16/16 ---
Parameters: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 2.2439

--- Grid Search Finished ---
Best parameters found: {'bidirectional': True, 'dropout': 0.1, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}
Best validation loss: 2.2377

Grid Search Results:
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}, 'val_loss': 2.588010787963867, 'val_gaze_accuracy': np.float64(0.21033653846153846), 'val_gaze_precision': 0.04424145941198225, 'val_gaze_f1': 0.07310604613856848, 'val_laughter_accuracy': np.float64(0.8221153846153846), 'val_laughter_precision': 0.8258505717456193, 'val_laughter_f1': 0.8239700127811396}
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}, 'val_loss': 2.515366554260254, 'val_gaze_accuracy': np.float64(0.21033653846153846), 'val_gaze_precision': 0.04424145941198225, 'val_ga

,params,val_loss,val_gaze_accuracy,val_gaze_precision,val_gaze_f1,val_laughter_accuracy,val_laughter_precision,val_laughter_f1
0,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.588011,0.210337,0.044241,0.073106,0.822115,0.825851,0.823970
1,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.515367,0.210337,0.044241,0.073106,0.900240,0.810433,0.852979
2,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.509538,0.205529,0.117012,0.077904,0.721154,0.866108,0.773109
3,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.539783,0.210337,0.044241,0.073106,0.663462,0.863208,0.730257
4,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.400160,0.210337,0.044241,0.073106,0.759615,0.844305,0.795022
5,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.618210,0.210337,0.044241,0.073106,0.900240,0.810433,0.852979
6,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.553844,0.210337,0.044241,0.073106,0.731971,0.832037,0.774188
7,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.638490,0.210337,0.044241,0.073106,0.668269,0.863841,0.733951
8,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",2.626694,0.210337,0.044241,0.073106,0.712740,0.869773,0.767452
9,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",2.544311,0.210337,0.044241,0.073106,0.633413,0.861000,0.706824


In [ ]:
# Define dataset parameters
dataset_params = DatasetParams(
    isGazeRelation=False,
    isMixedPhases=True,
    train_size=0.7,
    val_size=0.15,
    n_parts=4
)

# Run the grid search
results2, best_params = grid_search(
    model=EnfantReactionPredictor,
    dataset_params=dataset_params,
    param_grid=param_grid
)

print("\nGrid Search Results:")
for res in results2:
      print(res)

results_df_2 = pd.DataFrame(results2)

print("\nGrid Search Results DataFrame:")
display(results_df_2)

results_df_2.to_csv('grid_search_results_results_df_2.csv', index=False)

Starting Grid Search with 16 combinations...


Processing params:   0%|          | 0/16 [00:00<?, ?set/s]


--- Running combination 1/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

...

Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 2.7535

--- Running combination 16/16 ---
Parameters: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 2.9262

--- Grid Search Finished ---
Best parameters found: {'bidirectional': False, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}
Best validation loss: 2.5810

Grid Search Results:
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}, 'val_loss': 2.6145325303077698, 'val_gaze_accuracy': np.float64(0.4801587301587302), 'val_gaze_precision': 0.29493779493779493, 'val_gaze_f1': 0.3654176028605252, 'val_laughter_accuracy': np.float64(0.5317460317460317), 'val_laughter_precision': 0.6370628590940292, 'val_laughter_f1': 0.5234620163855016}
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}, 'val_loss': 2.79762727022171, 'val_gaze_accuracy': np.float64(0.4801587301587302), 'val_gaze_precision': 0.29493779493779493, 'val_gaze

,params,val_loss,val_gaze_accuracy,val_gaze_precision,val_gaze_f1,val_laughter_accuracy,val_laughter_precision,val_laughter_f1
0,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.614533,0.480159,0.294938,0.365418,0.531746,0.637063,0.523462
1,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.797627,0.480159,0.294938,0.365418,0.513889,0.549795,0.522340
2,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.653573,0.478175,0.294442,0.364462,0.583333,0.642361,0.588371
3,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.750827,0.480159,0.294938,0.365418,0.531746,0.568271,0.539869
4,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.691544,0.480159,0.294938,0.365418,0.529762,0.613801,0.527960
5,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.790794,0.456349,0.288829,0.353759,0.515873,0.513696,0.514755
6,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.581034,0.464286,0.301484,0.365579,0.571429,0.623884,0.577490
7,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.782799,0.480159,0.294938,0.365418,0.563492,0.637369,0.565589
8,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",2.774366,0.480159,0.294938,0.365418,0.480159,0.502502,0.488149
9,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",2.687032,0.478175,0.294442,0.364462,0.539683,0.589516,0.546394


In [ ]:
# Define dataset parameters
dataset_params = DatasetParams(
    isGazeRelation=False,
    isMixedPhases=True,
    train_size=0.6,
    val_size=0.2,
    n_parts=8
)

# Run the grid search
results3, best_params = grid_search(
    model=EnfantReactionPredictor,
    dataset_params=dataset_params,
    param_grid=param_grid
)

print("\nGrid Search Results:")
for res in results3:
      print(res)

results_df_3 = pd.DataFrame(results3)

print("\nGrid Search Results DataFrame:")
display(results_df_3)

results_df_3.to_csv('grid_search_lstm_results_df_3.csv', index=False)

Starting Grid Search with 16 combinations...


Processing params:   0%|          | 0/16 [00:00<?, ?set/s]


--- Running combination 1/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(


Training Epochs (Params {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs':…

...

Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 2.4284

--- Running combination 16/16 ---
Parameters: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 2.4370

--- Grid Search Finished ---
Best parameters found: {'bidirectional': False, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}
Best validation loss: 2.2812

Grid Search Results:
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}, 'val_loss': 2.3146994246376886, 'val_gaze_accuracy': np.float64(0.4444444444444444), 'val_gaze_precision': 0.5850590748228544, 'val_gaze_f1': 0.38772678863369714, 'val_laughter_accuracy': np.float64(0.7087087087087087), 'val_laughter_precision': 0.6983360349333422, 'val_laughter_f1': 0.7024105186267349}
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}, 'val_loss': 2.5036844412485757, 'val_gaze_accuracy': np.float64(0.44144144144144143), 'val_gaze_precision': 0.3457292772798003, 'val_ga

,params,val_loss,val_gaze_accuracy,val_gaze_precision,val_gaze_f1,val_laughter_accuracy,val_laughter_precision,val_laughter_f1
0,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.314699,0.444444,0.585059,0.387727,0.708709,0.698336,0.702411
1,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.503684,0.441441,0.345729,0.377085,0.648649,0.698922,0.662492
2,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.404988,0.409910,0.409225,0.384107,0.729730,0.717289,0.721221
3,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.496511,0.429429,0.334164,0.365368,0.716216,0.734146,0.722768
4,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.433504,0.444444,0.585059,0.387727,0.722222,0.709534,0.713763
5,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.393596,0.429429,0.334233,0.365371,0.702703,0.737162,0.712842
6,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.392797,0.436937,0.576842,0.382109,0.732733,0.721100,0.724866
7,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.281236,0.445946,0.585070,0.390784,0.717718,0.735086,0.724103
8,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",2.394277,0.427928,0.446538,0.379630,0.729730,0.717289,0.721221
9,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",2.312670,0.438438,0.447631,0.388239,0.686186,0.704179,0.693136


In [ ]:
# Define dataset parameters
dataset_params = DatasetParams(
    isGazeRelation=False,
    isMixedPhases=True,
    train_size=0.5,
    val_size=0.25,
    n_parts=8
)

# Run the grid search
results4, best_params = grid_search(
    model=EnfantReactionPredictor,
    dataset_params=dataset_params,
    param_grid=param_grid
)

print("\nGrid Search Results:")
for res in results4:
      print(res)

results_df_4 = pd.DataFrame(results4)

print("\nGrid Search Results DataFrame:")
display(results_df_4)

results_df_4.to_csv('grid_search_lstm_results_df_4.csv', index=False)

Starting Grid Search with 16 combinations...


Processing params:   0%|          | 0/16 [00:00<?, ?set/s]


--- Running combination 1/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(


Training Epochs (Params {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs':…

...

Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 2.4862

--- Running combination 16/16 ---
Parameters: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 2.3919

--- Grid Search Finished ---
Best parameters found: {'bidirectional': True, 'dropout': 0.1, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}
Best validation loss: 2.3437

Grid Search Results:
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}, 'val_loss': 2.463307817776998, 'val_gaze_accuracy': np.float64(0.47324613555291317), 'val_gaze_precision': 0.5812438182410051, 'val_gaze_f1': 0.4647774190930624, 'val_laughter_accuracy': np.float64(0.7098692033293698), 'val_laughter_precision': 0.7025515733461122, 'val_laughter_f1': 0.7048583508307874}
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}, 'val_loss': 2.6082361539204917, 'val_gaze_accuracy': np.float64(0.3947681331747919), 'val_gaze_precision': 0.5426136371100422, 'val_gaze_

,params,val_loss,val_gaze_accuracy,val_gaze_precision,val_gaze_f1,val_laughter_accuracy,val_laughter_precision,val_laughter_f1
0,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.463308,0.473246,0.581244,0.464777,0.709869,0.702552,0.704858
1,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.608236,0.394768,0.542614,0.379629,0.669441,0.672203,0.670729
2,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.356483,0.474435,0.600797,0.463512,0.633769,0.626774,0.629768
3,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.542779,0.453032,0.604364,0.433592,0.667063,0.686444,0.673215
4,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.687568,0.480380,0.585527,0.469849,0.653983,0.642203,0.646219
5,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.797011,0.480380,0.624043,0.468966,0.669441,0.690937,0.675916
6,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.582023,0.474435,0.600797,0.463512,0.693222,0.689871,0.691353
7,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.717262,0.362663,0.418346,0.365852,0.733650,0.732798,0.733207
8,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",2.444217,0.476813,0.602875,0.465813,0.671819,0.657162,0.660474
9,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",2.560667,0.467301,0.587951,0.458191,0.636147,0.633793,0.634914


In [ ]:
# Define dataset parameters
dataset_params = DatasetParams(
    isGazeRelation=False,
    isMixedPhases=True,
    train_size=0.5,
    val_size=0.25,
    n_parts=12
)

# Run the grid search
results5, best_params = grid_search(
    model=EnfantReactionPredictor,
    dataset_params=dataset_params,
    param_grid=param_grid
)

print("\nGrid Search Results:")
for res in results5:
      print(res)

results_df_5 = pd.DataFrame(results5)

print("\nGrid Search Results DataFrame:")
display(results_df_5)

results_df_5.to_csv('grid_search_lstm_results_df_5.csv', index=False)

Starting Grid Search with 16 combinations...


Processing params:   0%|          | 0/16 [00:00<?, ?set/s]


--- Running combination 1/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(


Training Epochs (Params {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs':…

...

Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 2.1169

--- Running combination 16/16 ---
Parameters: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 2.0780

--- Grid Search Finished ---
Best parameters found: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}
Best validation loss: 2.0550

Grid Search Results:
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}, 'val_loss': 2.0549972378290615, 'val_gaze_accuracy': np.float64(0.4482758620689655), 'val_gaze_precision': 0.23489907885051511, 'val_gaze_f1': 0.29261799729165805, 'val_laughter_accuracy': np.float64(0.7883472057074911), 'val_laughter_precision': 0.7855843210527784, 'val_laughter_f1': 0.7863923878580933}
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}, 'val_loss': 2.1666261783012977, 'val_gaze_accuracy': np.float64(0.4637336504161712), 'val_gaze_precision': 0.35804430088952777, 'val_g

,params,val_loss,val_gaze_accuracy,val_gaze_precision,val_gaze_f1,val_laughter_accuracy,val_laughter_precision,val_laughter_f1
0,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.054997,0.448276,0.234899,0.292618,0.788347,0.785584,0.786392
1,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.166626,0.463734,0.358044,0.393940,0.802616,0.803240,0.802909
2,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.133481,0.462545,0.446156,0.319687,0.778835,0.775878,0.776792
3,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.230139,0.463734,0.358044,0.393940,0.812128,0.811587,0.811838
4,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.106021,0.448276,0.234899,0.292618,0.808561,0.805631,0.804764
5,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.168642,0.463734,0.358044,0.393940,0.808561,0.810001,0.809185
6,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.143141,0.462545,0.446156,0.319687,0.800238,0.800238,0.800238
7,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.127059,0.478002,0.558595,0.422444,0.804994,0.803686,0.804220
8,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",2.083819,0.462545,0.446156,0.319687,0.815696,0.813239,0.813558
9,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",2.140528,0.478002,0.558595,0.422444,0.807372,0.807372,0.807372


### Results - with Gaze Relation

In [ ]:
# Define the parameter grid
param_grid = {
    'hidden_dim': [128, 256],
    'num_layers': [1, 2],
    'dropout': [0.1, 0.3],
    'bidirectional': [False, True],
    'lr': [0.003],
    'weight_decay': [1e-5],
    'num_epochs': [10]
}

In [ ]:
# Define dataset parameters
dataset_params = DatasetParams(
    isGazeRelation=True,
    isMixedPhases=False,
    train_size=0.5,
    val_size=0.25,
    n_parts=1
)

# Run the grid search
results_gr, best_params = grid_search(
    model=EnfantReactionPredictor,
    dataset_params=dataset_params,
    param_grid=param_grid
)

print("\nGrid Search Results:")
for res in results_gr:
      print(res)

results_gr_df = pd.DataFrame(results_gr)

print("\nGrid Search Results DataFrame:")
display(results_gr_df)
results_gr_df.to_csv('grid_search_lstm_results_gr_df.csv', index=False)

Starting Grid Search with 16 combinations...


Processing params:   0%|          | 0/16 [00:00<?, ?set/s]


--- Running combination 1/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(


Training Epochs (Params {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs':…

...

Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 2.5340

--- Running combination 16/16 ---
Parameters: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 2.4818

--- Grid Search Finished ---
Best parameters found: {'bidirectional': True, 'dropout': 0.1, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}
Best validation loss: 2.3890

Grid Search Results:
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}, 'val_loss': 2.662733554840088, 'val_gaze_accuracy': np.float64(0.21033653846153846), 'val_gaze_precision': 0.04424145941198225, 'val_gaze_f1': 0.07310604613856848, 'val_laughter_accuracy': np.float64(0.90625), 'val_laughter_precision': 0.9150922007255139, 'val_laughter_f1': 0.8670216601292109}
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}, 'val_loss': 2.7472763061523438, 'val_gaze_accuracy': np.float64(0.21033653846153846), 'val_gaze_precision': 0.04424145941198225, 'val_gaze_f1': 0.

,params,val_loss,val_gaze_accuracy,val_gaze_precision,val_gaze_f1,val_laughter_accuracy,val_laughter_precision,val_laughter_f1
0,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.662734,0.210337,0.044241,0.073106,0.906250,0.915092,0.867022
1,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.747276,0.210337,0.044241,0.073106,0.900240,0.810433,0.852979
2,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.959392,0.210337,0.044241,0.073106,0.743990,0.850859,0.786392
3,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",3.095035,0.210337,0.044241,0.073106,0.900240,0.810433,0.852979
4,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.584668,0.210337,0.044241,0.073106,0.846154,0.827863,0.836630
5,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.679430,0.210337,0.044241,0.073106,0.900240,0.810433,0.852979
6,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.846036,0.210337,0.044241,0.073106,0.858173,0.851922,0.854959
7,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",3.128105,0.210337,0.044241,0.073106,0.884615,0.809007,0.845124
8,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",2.768750,0.210337,0.044241,0.073106,0.846154,0.866025,0.855212
9,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",2.979787,0.210337,0.044241,0.073106,0.795673,0.859958,0.822297


In [ ]:
# Define dataset parameters
dataset_params = DatasetParams(
    isGazeRelation=True,
    isMixedPhases=True,
    train_size=0.7,
    val_size=0.15,
    n_parts=4
)

# Run the grid search
results_gr2, best_params = grid_search(
    model=EnfantReactionPredictor,
    dataset_params=dataset_params,
    param_grid=param_grid
)

print("\nGrid Search Results:")
for res in results_gr2:
      print(res)

results_gr_df_2 = pd.DataFrame(results_gr2)

print("\nGrid Search Results DataFrame:")
display(results_gr_df_2)

results_gr_df_2.to_csv('grid_search_lstm_results_gr_df_2.csv', index=False)

Starting Grid Search with 16 combinations...


Processing params:   0%|          | 0/16 [00:00<?, ?set/s]


--- Running combination 1/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(


Training Epochs (Params {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs':…

...

Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 2.9037

--- Running combination 16/16 ---
Parameters: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 2.7746

--- Grid Search Finished ---
Best parameters found: {'bidirectional': False, 'dropout': 0.3, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}
Best validation loss: 2.5125

Grid Search Results:
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}, 'val_loss': 2.66377529501915, 'val_gaze_accuracy': np.float64(0.45634920634920634), 'val_gaze_precision': 0.2888286116134217, 'val_gaze_f1': 0.3537590746893073, 'val_laughter_accuracy': np.float64(0.5853174603174603), 'val_laughter_precision': 0.6518305902363873, 'val_laughter_f1': 0.5891294488810017}
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}, 'val_loss': 2.6496882140636444, 'val_gaze_accuracy': np.float64(0.4781746031746032), 'val_gaze_precision': 0.29960814735250824, 'val_gaze

,params,val_loss,val_gaze_accuracy,val_gaze_precision,val_gaze_f1,val_laughter_accuracy,val_laughter_precision,val_laughter_f1
0,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.663775,0.456349,0.288829,0.353759,0.585317,0.651831,0.589129
1,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.649688,0.478175,0.299608,0.368393,0.490079,0.505653,0.496337
2,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.670262,0.480159,0.294938,0.365418,0.539683,0.589516,0.546394
3,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.582662,0.476190,0.294801,0.364053,0.605159,0.664645,0.610044
4,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.750413,0.480159,0.294938,0.365418,0.539683,0.589516,0.546394
5,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.512468,0.468254,0.297871,0.364117,0.480159,0.474181,0.477002
6,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.708717,0.480159,0.294938,0.365418,0.488095,0.550940,0.491427
7,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.700618,0.480159,0.294938,0.365418,0.527778,0.565403,0.535924
8,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",2.776548,0.480159,0.294938,0.365418,0.474206,0.495141,0.481979
9,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",2.628343,0.480159,0.294938,0.365418,0.545635,0.564339,0.552154


In [ ]:
# Define dataset parameters
dataset_params = DatasetParams(
    isGazeRelation=True,
    isMixedPhases=True,
    train_size=0.6,
    val_size=0.2,
    n_parts=8
)

# Run the grid search
results_gr3, best_params = grid_search(
    model=EnfantReactionPredictor,
    dataset_params=dataset_params,
    param_grid=param_grid
)

print("\nGrid Search Results:")
for res in results_gr3:
      print(res)

results_gr_df_3 = pd.DataFrame(results_gr3)

print("\nGrid Search Results DataFrame:")
display(results_gr_df_3)

results_gr_df_3.to_csv('grid_search_lstm_results_gr_df_3.csv', index=False)

Starting Grid Search with 16 combinations...


Processing params:   0%|          | 0/16 [00:00<?, ?set/s]


--- Running combination 1/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(


Training Epochs (Params {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs':…

...

Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 2.4559

--- Running combination 16/16 ---
Parameters: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 2.4436

--- Grid Search Finished ---
Best parameters found: {'bidirectional': True, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}
Best validation loss: 2.2301

Grid Search Results:
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}, 'val_loss': 2.260222143597073, 'val_gaze_accuracy': np.float64(0.45045045045045046), 'val_gaze_precision': 0.3549148431085453, 'val_gaze_f1': 0.38568788244111496, 'val_laughter_accuracy': np.float64(0.7282282282282282), 'val_laughter_precision': 0.7165884447321573, 'val_laughter_f1': 0.7205026585831539}
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}, 'val_loss': 2.3666918807559543, 'val_gaze_accuracy': np.float64(0.4174174174174174), 'val_gaze_precision': 0.31977011111708464, 'val_gaz

,params,val_loss,val_gaze_accuracy,val_gaze_precision,val_gaze_f1,val_laughter_accuracy,val_laughter_precision,val_laughter_f1
0,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.260222,0.450450,0.354915,0.385688,0.728228,0.716588,0.720503
1,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.366692,0.417417,0.319770,0.358537,0.680180,0.711525,0.690377
2,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.454218,0.445946,0.585823,0.391005,0.732733,0.721100,0.724866
3,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.285018,0.436937,0.576842,0.382109,0.758258,0.749576,0.752340
4,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.311449,0.424925,0.337656,0.363874,0.723724,0.712756,0.716691
5,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.538424,0.418919,0.287983,0.326854,0.659159,0.704596,0.672107
6,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.396811,0.436937,0.576842,0.382109,0.729730,0.717289,0.721221
7,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.391239,0.429429,0.335556,0.365651,0.711712,0.730320,0.718501
8,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",2.328150,0.436937,0.576710,0.382148,0.716216,0.705858,0.709812
9,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",2.230053,0.430931,0.573681,0.368786,0.690691,0.693422,0.692001


In [ ]:
# Define dataset parameters
dataset_params = DatasetParams(
    isGazeRelation=True,
    isMixedPhases=True,
    train_size=0.5,
    val_size=0.25,
    n_parts=8
)

# Run the grid search
results_gr4, best_params = grid_search(
    model=EnfantReactionPredictor,
    dataset_params=dataset_params,
    param_grid=param_grid
)

print("\nGrid Search Results:")
for res in results_gr4:
      print(res)

results_gr_df_4 = pd.DataFrame(results_gr4)

print("\nGrid Search Results DataFrame:")
display(results_gr_df_4)

results_gr_df_4.to_csv('grid_search_lstm_results_gr_df_4.csv', index=False)

Starting Grid Search with 16 combinations...


Processing params:   0%|          | 0/16 [00:00<?, ?set/s]


--- Running combination 1/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(


Training Epochs (Params {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs':…

...

Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 2.4147

--- Running combination 16/16 ---
Parameters: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 2.5360

--- Grid Search Finished ---
Best parameters found: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}
Best validation loss: 2.2919

Grid Search Results:
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}, 'val_loss': 2.29189493921068, 'val_gaze_accuracy': np.float64(0.4221165279429251), 'val_gaze_precision': 0.5074975045311051, 'val_gaze_f1': 0.43475880210412604, 'val_laughter_accuracy': np.float64(0.7253269916765755), 'val_laughter_precision': 0.7155413633280076, 'val_laughter_f1': 0.7160183765299343}
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}, 'val_loss': 2.392394688394335, 'val_gaze_accuracy': np.float64(0.3971462544589774), 'val_gaze_precision': 0.4339919773394563, 'val_gaze_f

,params,val_loss,val_gaze_accuracy,val_gaze_precision,val_gaze_f1,val_laughter_accuracy,val_laughter_precision,val_laughter_f1
0,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.291895,0.422117,0.507498,0.434759,0.725327,0.715541,0.716018
1,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.392395,0.397146,0.433992,0.394366,0.724138,0.725056,0.724580
2,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.365558,0.472057,0.579911,0.463544,0.697979,0.691331,0.693744
3,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.603810,0.382878,0.449359,0.388111,0.683710,0.697853,0.688595
4,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.407892,0.458977,0.520847,0.466226,0.707491,0.700426,0.702761
5,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.359351,0.370987,0.396330,0.355686,0.730083,0.729864,0.729973
6,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.369983,0.470868,0.579166,0.462477,0.668252,0.667445,0.667840
7,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.510900,0.369798,0.437684,0.378583,0.702735,0.697411,0.699487
8,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",2.538707,0.473246,0.581244,0.464777,0.664685,0.649803,0.653548
9,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",2.589718,0.385256,0.430650,0.381069,0.695600,0.695107,0.695350


In [ ]:
# Define dataset parameters
dataset_params = DatasetParams(
    isGazeRelation=True,
    isMixedPhases=True,
    train_size=0.5,
    val_size=0.25,
    n_parts=12
)

# Run the grid search
results_gr5, best_params = grid_search(
    model=EnfantReactionPredictor,
    dataset_params=dataset_params,
    param_grid=param_grid
)

print("\nGrid Search Results:")
for res in results_gr5:
      print(res)

results_gr_df_5 = pd.DataFrame(results_gr5)

print("\nGrid Search Results DataFrame:")
display(results_gr_df_5)

results_gr_df_5.to_csv('grid_search_lstm_results_gr_df_5.csv', index=False)

Starting Grid Search with 16 combinations...


Processing params:   0%|          | 0/16 [00:00<?, ?set/s]


--- Running combination 1/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn(


Training Epochs (Params {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs':…

...

Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 2.1215

--- Running combination 16/16 ---
Parameters: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 2.1198

--- Grid Search Finished ---
Best parameters found: {'bidirectional': False, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}
Best validation loss: 2.0515

Grid Search Results:
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}, 'val_loss': 2.130355899150555, 'val_gaze_accuracy': np.float64(0.4482758620689655), 'val_gaze_precision': 0.23489907885051511, 'val_gaze_f1': 0.29261799729165805, 'val_laughter_accuracy': np.float64(0.7835909631391201), 'val_laughter_precision': 0.7809281613766356, 'val_laughter_f1': 0.7817905438739964}
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}, 'val_loss': 2.0703148337510915, 'val_gaze_accuracy': np.float64(0.4637336504161712), 'val_gaze_precision': 0.3569404096125862, 'val_gaz

,params,val_loss,val_gaze_accuracy,val_gaze_precision,val_gaze_f1,val_laughter_accuracy,val_laughter_precision,val_laughter_f1
0,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.130356,0.448276,0.234899,0.292618,0.783591,0.780928,0.781791
1,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.070315,0.463734,0.356940,0.392877,0.795482,0.794888,0.795166
2,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.134282,0.462545,0.446156,0.319687,0.788347,0.787442,0.787852
3,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.082203,0.463734,0.358044,0.393940,0.807372,0.811378,0.808809
4,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.142583,0.448276,0.234899,0.292618,0.788347,0.785584,0.786392
5,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.132588,0.463734,0.356940,0.392877,0.793103,0.793103,0.793103
6,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.051510,0.448276,0.234899,0.292618,0.793103,0.791696,0.792282
7,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.089095,0.463734,0.358044,0.393940,0.789536,0.788778,0.789128
8,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",2.089501,0.462545,0.446156,0.319687,0.780024,0.777863,0.778683
9,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",2.077789,0.463734,0.358044,0.393940,0.807372,0.810949,0.808692


## Training Best Model

In [ ]:
dataset_params = DatasetParams(
    isGazeRelation=False,
    isMixedPhases=True,
    train_size=0.5,
    val_size=0.25,
    n_parts=12
)

model_params = ModelParams(
    hidden_dim=128,
    bidirectional=True,
    num_layers=2,
    dropout=0.1,
    input_size=416
)

train_params = TrainParams(
    lr = 0.003,
    weight_decay = 1e-5,
    num_epochs = 25
)

model = training_model(EnfantReactionPredictor, dataset_params, model_params, train_params)

Starting training...


Training Epochs:   0%|          | 0/25 [00:00<?, ?it/s]

Epoch [1/25], 	Train Loss: 1.6386, Train Gaze Acc: 0.5649, Train Gaze Prec: 0.3191, Train Gaze F1: 0.4078, Train Laughter Acc: 0.7333, Train Laughter Prec: 0.7218, Train Laughter F1: 0.7233, 
	Val Loss: 1.8908, Val Gaze Acc: 0.4819, Val Gaze Prec: 0.2322, Val Gaze F1: 0.3134, Val Laughter Acc: 0.7089, Val Laughter Prec: 0.7029, Val Laughter F1: 0.7024
Epoch [2/25], 	Train Loss: 1.4011, Train Gaze Acc: 0.6047, Train Gaze Prec: 0.5558, Train Gaze F1: 0.5582, Train Laughter Acc: 0.7471, Train Laughter Prec: 0.7779, Train Laughter F1: 0.7541, 
	Val Loss: 1.8580, Val Gaze Acc: 0.5592, Val Gaze Prec: 0.4302, Val Gaze F1: 0.4727, Val Laughter Acc: 0.6860, Val Laughter Prec: 0.7017, Val Laughter F1: 0.6898
Epoch [3/25], 	Train Loss: 1.3547, Train Gaze Acc: 0.6439, Train Gaze Prec: 0.6851, Train Gaze F1: 0.6258, Train Laughter Acc: 0.7568, Train Laughter Prec: 0.7591, Train Laughter F1: 0.7579, 
	Val Loss: 1.8316, Val Gaze Acc: 0.5785, Val Gaze Prec: 0.5323, Val Gaze F1: 0.5063, Val Laughter Ac

### 10-fold CV

In [ ]:
def get_targets(y):
    target_laughter_indices = []
    target_gaze_indices = []

    for target_dict in y:
        laughter_val = target_dict.get('Laughter@CHI')
        gaze_val = target_dict.get('Gaze@CHI')

        target_laughter_indices.append(lookup(vocab_laughter_chi, laughter_val))
        target_gaze_indices.append(lookup(vocab_gaze_chi, gaze_val))

    targets_laughter = torch.tensor(target_laughter_indices, dtype=torch.long)
    targets_gaze = torch.tensor(target_gaze_indices, dtype=torch.long)

    return targets_gaze, targets_laughter

In [ ]:
from sklearn.model_selection import KFold

def CrossValidation(dataset_params: DatasetParams, model_params: ModelParams, train_params: TrainParams):
    # Combine train and test data for cross-validation
    X_combined, vocabs = get_data(dataset_params.isGazeRelation)

    kf = KFold(n_splits=10, shuffle=False)

    test_losses = []
    test_gaze_accuracies = []
    test_laughter_accuracies = []

    print("Starting 10-Fold Cross-Validation...")

    # Iterate over each fold
    for fold, (train_index, test_index) in tqdm(enumerate(kf.split(X_combined)), total=kf.get_n_splits(), desc="Cross-Validation Progress"):
        print(f"\n--- Fold {fold+1}/10 ---")

        # Split data into train and test sets for the current fold
        train_fold = [X_combined[i] for i in train_index]
        test_fold = [X_combined[i] for i in test_index]

        X_train_fold, y_train_fold = DatasetSplit(dataset = X, n_parts = 4, train_size = 0.7, val_size = 0).get_features_and_targets(train_fold)
        X_test_fold, y_test_fold = DatasetSplit(dataset = X, n_parts = 4, train_size = 0.7, val_size = 0).get_features_and_targets(test_fold)

        y_train_fold_gaze, y_train_fold_laughter = get_targets(y_train_fold)
        y_test_fold_gaze, y_test_fold_laughter = get_targets(y_test_fold)

        model = EnfantReactionPredictor(
            vocabs,
            dataset_params.isGazeRelation,
            model_params.hidden_dim,
            model_params.num_layers,
            model_params.dropout,
            model_params.input_size,
            model_params.bidirectional
        ).to(device)

        optimizer = optim.Adam(model.parameters(), lr=0.003, weight_decay=1e-5)
        criterion = nn.CrossEntropyLoss()

        num_epochs_fold = 10
        for epoch in range(num_epochs_fold):
            model.train()
            (logits_gaze, logits_laughter) = model([X_train_fold], device)
            loss_gaze = criterion(logits_gaze.squeeze(0), y_train_fold_gaze)
            loss_laughter = criterion(logits_laughter.squeeze(0), y_train_fold_laughter)
            loss = loss_gaze + loss_laughter

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Evaluate on the test set for the current fold
        model.eval()
        with torch.no_grad():
            (test_logits_gaze, test_logits_laughter) = model([X_test_fold], device)

            test_loss_gaze = criterion(test_logits_gaze.squeeze(0), y_test_fold_gaze)
            test_loss_laughter = criterion(test_logits_laughter.squeeze(0), y_test_fold_laughter)
            test_total_loss = test_loss_gaze + test_loss_laughter

            test_pred_gaze = torch.argmax(test_logits_gaze.squeeze(0), dim=1)
            test_correct_gaze = (test_pred_gaze == y_test_fold_gaze).sum().item()
            test_accuracy_gaze = test_correct_gaze / y_test_fold_gaze.size(0)

            test_pred_laughter = torch.argmax(test_logits_laughter.squeeze(0), dim=1)
            test_correct_laughter = (test_pred_laughter == y_test_fold_laughter).sum().item()
            test_accuracy_laughter = test_correct_laughter / y_test_fold_laughter.size(0)


        test_losses.append(test_total_loss.item())
        test_gaze_accuracies.append(test_accuracy_gaze)
        test_laughter_accuracies.append(test_accuracy_laughter)

        print(f"Fold {fold+1} - Test Loss: {test_total_loss:.4f}, Test Gaze Acc: {test_accuracy_gaze:.4f}, Test Laughter Acc: {test_accuracy_laughter:.4f}")

    # average performance across all folds
    print("\n--- Cross-Validation Results ---")
    print(f"Average Test Loss: {np.mean(test_losses):.4f}")
    print(f"Average Test Gaze Accuracy: {np.mean(test_gaze_accuracies):.4f}")
    print(f"Average Test Laughter Accuracy: {np.mean(test_laughter_accuracies):.4f}")

In [ ]:
dataset_params = DatasetParams(
    isGazeRelation=False,
    isMixedPhases=True,
    train_size=0.5,
    val_size=0.25,
    n_parts=12
)

model_params = ModelParams(
    hidden_dim=128,
    bidirectional=True,
    num_layers=2,
    dropout=0.1,
    input_size=416
)

train_params = TrainParams(
    lr = 0.003,
    weight_decay = 1e-5,
    num_epochs = 25
)

CrossValidation(dataset_params, model_params, train_params)

Starting 10-Fold Cross-Validation...


Cross-Validation Progress:   0%|          | 0/10 [00:00<?, ?it/s]


--- Fold 1/10 ---
Train Fold Size: 2993, Test Fold Size: 333
X Train Fold Size: 2993, Y Train Fold Size: 2993
X Test Fold Size: 333, Y Test Fold Size: 333
Fold 1 - Test Loss: 1.6578, Test Gaze Acc: 0.5706, Test Laughter Acc: 0.5916

--- Fold 2/10 ---
Train Fold Size: 2993, Test Fold Size: 333
X Train Fold Size: 2993, Y Train Fold Size: 2993
X Test Fold Size: 333, Y Test Fold Size: 333
Fold 2 - Test Loss: 1.2986, Test Gaze Acc: 0.8438, Test Laughter Acc: 0.6997

--- Fold 3/10 ---
Train Fold Size: 2993, Test Fold Size: 333
X Train Fold Size: 2993, Y Train Fold Size: 2993
X Test Fold Size: 333, Y Test Fold Size: 333
Fold 3 - Test Loss: 1.4585, Test Gaze Acc: 0.6637, Test Laughter Acc: 0.6667

--- Fold 4/10 ---
Train Fold Size: 2993, Test Fold Size: 333
X Train Fold Size: 2993, Y Train Fold Size: 2993
X Test Fold Size: 333, Y Test Fold Size: 333
Fold 4 - Test Loss: 2.3247, Test Gaze Acc: 0.0300, Test Laughter Acc: 0.8619

--- Fold 5/10 ---
Train Fold Size: 2993, Test Fold Size: 333
X Trai

# Multi-LSTM

In [ ]:
class EnfantReactionPredictorMultiLSTM(nn.Module):
    def __init__(self,
                 # Vocabularies for embedding lookups
                 vocabs,
                 # Whether to include gaze relation prediction
                 isGazeRelation,
                 # LSTM hyperparameters
                 hidden_dim = 192,
                 num_layers = 2,
                 dropout = 0.3,
                 input_size = 40,
                 bidirectional = False
                 ):
        super().__init__()

        self.isGazeRelation = isGazeRelation

        # Store modality vocabularies
        self.vocab_gaze = vocabs["gaze"]
        self.vocab_utterance = vocabs["utterance"]
        self.vocab_prosody = vocabs["prosody"]
        self.vocab_facial = vocabs["facial"]
        self.vocab_laughter = vocabs["laughter"]
        self.vocab_gaze_chi = vocabs["gaze_chi"]
        self.vocab_laughter_chi = vocabs["laughter_chi"]

        if self.isGazeRelation:
            self.vocab_gazerelation = vocabs["gazerelation"]

        # Embedding layers for categorical inputs
        self.embed_gaze = nn.Embedding(len(self.vocab_gaze), 8)
        self.embed_utterance = nn.Embedding(len(self.vocab_utterance), 8)
        self.embed_prosody = nn.Embedding(len(self.vocab_prosody), 4)
        self.embed_facial = nn.Embedding(len(self.vocab_facial), 8)
        self.embed_laughter = nn.Embedding(len(self.vocab_laughter), 4)

        # Sentence transformer for action text encoding
        self.sentence_encoder = SentenceTransformer('all-MiniLM-L6-v2')

        # Embeddings for child laughter and gaze
        self.embed_laughter_chi = nn.Embedding(len(self.vocab_laughter_chi), 4)
        self.embed_gaze_chi = nn.Embedding(len(self.vocab_gaze_chi), 8)
        if self.isGazeRelation:
            self.embed_gazerelation = nn.Embedding(len(self.vocab_gazerelation), 4)

        # LSTM configuration for each modality
        self.hidden_dim = hidden_dim

        gaze_lstm_input_size = self.embed_gaze.embedding_dim
        utt_lstm_input_size = self.embed_utterance.embedding_dim
        prosody_lstm_input_size = self.embed_prosody.embedding_dim
        facial_lstm_input_size = self.embed_facial.embedding_dim
        laughter_lstm_input_size = self.embed_laughter.embedding_dim
        action_lstm_input_size = self.sentence_encoder.get_sentence_embedding_dimension()

        # One LSTM per modality
        self.lstm_gaze = nn.LSTM(gaze_lstm_input_size, hidden_dim, num_layers=num_layers, dropout=dropout, batch_first=True, bidirectional=bidirectional)
        self.lstm_utt = nn.LSTM(utt_lstm_input_size, hidden_dim, num_layers=num_layers, dropout=dropout, batch_first=True, bidirectional=bidirectional)
        self.lstm_prosody = nn.LSTM(prosody_lstm_input_size, hidden_dim, num_layers=num_layers, dropout=dropout, batch_first=True, bidirectional=bidirectional)
        self.lstm_facial = nn.LSTM(facial_lstm_input_size, hidden_dim, num_layers=num_layers, dropout=dropout, batch_first=True, bidirectional=bidirectional)
        self.lstm_laughter = nn.LSTM(laughter_lstm_input_size, hidden_dim, num_layers=num_layers, dropout=dropout, batch_first=True, bidirectional=bidirectional)
        self.lstm_action = nn.LSTM(action_lstm_input_size, hidden_dim, num_layers=num_layers, dropout=dropout, batch_first=True, bidirectional=bidirectional)

        # Fusion layer after concatenating all modality features
        fusion_input_size = hidden_dim * 6 * (2 if bidirectional else 1)
        self.fc_fusion = nn.Linear(fusion_input_size, hidden_dim)

        self.dropout = nn.Dropout(dropout)

        # Output layers for each prediction target
        self.decoder_laughter = nn.Linear(hidden_dim, len(self.vocab_laughter_chi))
        self.decoder_gaze = nn.Linear(hidden_dim, len(self.vocab_gaze_chi))
        if self.isGazeRelation:
            self.decoder_gazerelation = nn.Linear(hidden_dim, len(self.vocab_gazerelation))

    # Finds the index of a value in the vocabulary (or <UNK> if missing)
    def lookup(self, vocab, value):
        if value is None:
            value = "<UNK>"
        return vocab.get(value, vocab["<UNK>"])

    # Converts a batch of sequences of modality dictionaries into embeddings
    def encode_input_batch(self, batch_sequence_of_dicts, device):
        """
        Args:
            batch_sequence_of_dicts: List[List[Dict]]  # [batch_size, seq_len]
        Returns:
            tuple of modality tensors with shape [batch_size, seq_len, embedding_dim]
        """
        batch_size = len(batch_sequence_of_dicts)
        seq_len = len(batch_sequence_of_dicts[0])

        # Prepare lists to hold token indices/embeddings
        gaze_indices, utterance_indices, prosody_indices, facial_indices, laughter_indices, action_embeddings = [], [], [], [], [], []

        for sequence in batch_sequence_of_dicts:
            gaze_seq, utt_seq, prosody_seq, facial_seq, laughter_seq, action_seq = [], [], [], [], [], []

            for step_dict in sequence:
                gaze_seq.append(self.lookup(self.vocab_gaze, step_dict.get("Gaze@MOT")))
                utt_seq.append(self.lookup(self.vocab_utterance, step_dict.get("Utterance@MOT")))
                prosody_seq.append(self.lookup(self.vocab_prosody, step_dict.get("Prosody@MOT")))
                facial_seq.append(self.lookup(self.vocab_facial, step_dict.get("Facial@MOT")))
                laughter_seq.append(self.lookup(self.vocab_laughter, step_dict.get("Laughter@MOT")))

                action_text = step_dict.get("Action@MOT")
                if action_text is None or not isinstance(action_text, str):
                    action_text = "<UNK>"
                encoded = self.sentence_encoder.encode(action_text)  # Vector representation
                action_seq.append(torch.tensor(encoded))

            # Convert sequences to tensors
            gaze_indices.append(torch.tensor(gaze_seq))
            utterance_indices.append(torch.tensor(utt_seq))
            prosody_indices.append(torch.tensor(prosody_seq))
            facial_indices.append(torch.tensor(facial_seq))
            laughter_indices.append(torch.tensor(laughter_seq))
            action_embeddings.append(torch.stack(action_seq))

        # Embed categorical inputs and move to device
        gaze = self.embed_gaze(torch.stack(gaze_indices).to(device))
        utt = self.embed_utterance(torch.stack(utterance_indices).to(device))
        prosody = self.embed_prosody(torch.stack(prosody_indices).to(device))
        facial = self.embed_facial(torch.stack(facial_indices).to(device))
        laughter = self.embed_laughter(torch.stack(laughter_indices).to(device))
        action = torch.stack(action_embeddings).to(device)

        return gaze, utt, prosody, facial, laughter, action

    def forward(self, batch_sequence_of_dicts, device):
        """
        Forward pass:
        - Encodes each modality
        - Passes them through modality-specific LSTMs
        - Concatenates final hidden states
        - Passes through fusion + output layers
        """
        # Encode inputs
        gaze_emb, utt_emb, prosody_emb, facial_emb, laughter_emb, action_emb = self.encode_input_batch(batch_sequence_of_dicts, device)

        # Helper to get last hidden state from LSTM output
        def get_last_hidden(h_n, num_layers, bidirectional):
            if bidirectional:
                return torch.cat((h_n[-2, :, :], h_n[-1, :, :]), dim=1)
            else:
                return h_n[-1, :, :]

        # Process each modality with its LSTM and extract last hidden state
        _, (h_gaze, _) = self.lstm_gaze(gaze_emb)
        h_gaze_last = get_last_hidden(h_gaze, self.lstm_gaze.num_layers, self.lstm_gaze.bidirectional)

        _, (h_utt, _) = self.lstm_utt(utt_emb)
        h_utt_last = get_last_hidden(h_utt, self.lstm_utt.num_layers, self.lstm_utt.bidirectional)

        _, (h_prosody, _) = self.lstm_prosody(prosody_emb)
        h_prosody_last = get_last_hidden(h_prosody, self.lstm_prosody.num_layers, self.lstm_prosody.bidirectional)

        _, (h_facial, _) = self.lstm_facial(facial_emb)
        h_facial_last = get_last_hidden(h_facial, self.lstm_facial.num_layers, self.lstm_facial.bidirectional)

        _, (h_laughter, _) = self.lstm_laughter(laughter_emb)
        h_laughter_last = get_last_hidden(h_laughter, self.lstm_laughter.num_layers, self.lstm_laughter.bidirectional)

        _, (h_action, _) = self.lstm_action(action_emb)
        h_action_last = get_last_hidden(h_action, self.lstm_action.num_layers, self.lstm_action.bidirectional)

        # Concatenate final hidden states from all modalities
        h_final = torch.cat([h_gaze_last, h_utt_last, h_prosody_last, h_facial_last, h_laughter_last, h_action_last], dim=1)

        # Fusion layer + activation + dropout
        fused = self.dropout(torch.relu(self.fc_fusion(h_final)))

        # Prediction heads
        pred_laughter = self.decoder_laughter(fused)
        pred_gaze = self.decoder_gaze(fused)

        if self.isGazeRelation:
            pred_gazerelation = self.decoder_gazerelation(fused)
            return (pred_gaze, pred_laughter, pred_gazerelation)

        return (pred_gaze, pred_laughter)


## Grid Search Results

### Results - Standard

In [ ]:
# Define the parameter grid
param_grid = {
    'hidden_dim': [128, 256],
    'num_layers': [1, 2],
    'dropout': [0.1, 0.3],
    'bidirectional': [False, True],
    'lr': [0.003],
    'weight_decay': [1e-5],
    'num_epochs': [10]
}

In [ ]:
# Define dataset parameters
dataset_params = DatasetParams(
    isGazeRelation=False,
    isMixedPhases=False,
    train_size=0.5,
    val_size=0.25,
    n_parts=1
)

# Run the grid search
results, best_params = grid_search(
    model=EnfantReactionPredictorMultiLSTM,
    dataset_params=dataset_params,
    param_grid=param_grid
)

print("\nGrid Search Results:")
for res in results:
      print(res)

results_df = pd.DataFrame(results)

print("\nGrid Search Results DataFrame:")
display(results_df)

Starting Grid Search with 16 combinations...


Processing params:   0%|          | 0/16 [00:00<?, ?set/s]


--- Running combination 1/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 2.4058

--- Running combination 16/16 ---
Parameters: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 2.7609

--- Grid Search Finished ---
Best parameters found: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}
Best validation loss: 2.2037

Grid Search Results:
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}, 'val_loss': 2.6331043243408203, 'val_gaze_accuracy': np.float64(0.21033653846153846), 'val_gaze_precision': 0.04424145941198225, 'val_gaze_f1': 0.07310604613856848, 'val_laughter_accuracy': np.float64(0.7391826923076923), 'val_laughter_precision': 0.8332176087214584, 'val_laughter_f1': 0.7790850503929996}
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}, 'val_loss': 2.570742130279541, 'val_gaze_accuracy': np.float64(0.21033653846153846), 'val_gaze_precision': 0.04424145941198225, 'val_

,params,val_loss,val_gaze_accuracy,val_gaze_precision,val_gaze_f1,val_laughter_accuracy,val_laughter_precision,val_laughter_f1
0,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.633104,0.210337,0.044241,0.073106,0.739183,0.833218,0.779085
1,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.570742,0.210337,0.044241,0.073106,0.900240,0.810433,0.852979
2,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.203681,0.425481,0.215291,0.285911,0.814904,0.881533,0.840502
3,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.321635,0.210337,0.044241,0.073106,0.721154,0.866108,0.773109
4,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.563527,0.210337,0.044241,0.073106,0.750000,0.870339,0.794060
5,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.707820,0.210337,0.044241,0.073106,0.900240,0.810433,0.852979
6,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.237137,0.424279,0.214824,0.285199,0.842548,0.887696,0.860362
7,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.466729,0.210337,0.044241,0.073106,0.706731,0.872242,0.763285
8,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",2.327090,0.349760,0.179016,0.233334,0.673077,0.864475,0.737629
9,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",2.399076,0.210337,0.044241,0.073106,0.750000,0.870339,0.794060


In [ ]:
results_df.to_csv('grid_search_multilstm_results_df.csv', index=False)

In [ ]:
# Define dataset parameters
dataset_params = DatasetParams(
    isGazeRelation=False,
    isMixedPhases=True,
    train_size=0.7,
    val_size=0.15,
    n_parts=4
)

# Run the grid search
results2, best_params = grid_search(
    model=EnfantReactionPredictorMultiLSTM,
    dataset_params=dataset_params,
    param_grid=param_grid
)

print("\nGrid Search Results:")
for res in results2:
      print(res)

results_df_2 = pd.DataFrame(results2)

print("\nGrid Search Results DataFrame:")
display(results_df_2)

Starting Grid Search with 16 combinations...


Processing params:   0%|          | 0/16 [00:00<?, ?set/s]


--- Running combination 1/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs':…

Validation Loss for this combo: 2.8353

--- Running combination 2/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


...

Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 3.0016

--- Running combination 16/16 ---
Parameters: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 3.3279

--- Grid Search Finished ---
Best parameters found: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}
Best validation loss: 2.7475

Grid Search Results:
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}, 'val_loss': 2.8352612257003784, 'val_gaze_accuracy': np.float64(0.45634920634920634), 'val_gaze_precision': 0.2888286116134217, 'val_gaze_f1': 0.3537590746893073, 'val_laughter_accuracy': np.float64(0.5238095238095238), 'val_laughter_precision': 0.5625247703116556, 'val_laughter_f1': 0.5319634963443909}
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}, 'val_loss': 3.052247643470764, 'val_gaze_accuracy': np.float64(0.4801587301587302), 'val_gaze_precision': 0.30466924502457493, 'val_gaz

,params,val_loss,val_gaze_accuracy,val_gaze_precision,val_gaze_f1,val_laughter_accuracy,val_laughter_precision,val_laughter_f1
0,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.835261,0.456349,0.288829,0.353759,0.523810,0.562525,0.531963
1,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",3.052248,0.480159,0.304669,0.372794,0.478175,0.498034,0.485661
2,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.747478,0.464286,0.301484,0.365579,0.599206,0.677125,0.601132
3,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",3.359884,0.464286,0.301484,0.365579,0.553571,0.588445,0.561355
4,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.790398,0.480159,0.294938,0.365418,0.519841,0.536432,0.526138
5,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",3.287097,0.478175,0.312941,0.378303,0.535714,0.601860,0.539110
6,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",3.228071,0.480159,0.294938,0.365418,0.559524,0.612866,0.565547
7,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",3.390615,0.480159,0.301607,0.370493,0.587302,0.621599,0.594502
8,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",2.971865,0.480159,0.294938,0.365418,0.563492,0.605055,0.570814
9,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",3.592446,0.480159,0.300851,0.369922,0.585317,0.630550,0.592020


In [ ]:
results_df_2.to_csv('grid_search_multilstm_results_df_2.csv', index=False)


In [ ]:
# Define dataset parameters
dataset_params = DatasetParams(
    isGazeRelation=False,
    isMixedPhases=True,
    train_size=0.6,
    val_size=0.2,
    n_parts=8
)

# Run the grid search
results3, best_params = grid_search(
    model=EnfantReactionPredictorMultiLSTM,
    dataset_params=dataset_params,
    param_grid=param_grid
)

print("\nGrid Search Results:")
for res in results3:
      print(res)

results_df_3 = pd.DataFrame(results3)

print("\nGrid Search Results DataFrame:")
display(results_df_3)

results_df_3.to_csv('grid_search_multilstm_results_df_3.csv', index=False)


Starting Grid Search with 16 combinations...


Processing params:   0%|          | 0/16 [00:00<?, ?set/s]


--- Running combination 1/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs':…

Validation Loss for this combo: 2.4179

--- Running combination 2/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


...

Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 2.4532

--- Running combination 16/16 ---
Parameters: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 2.3582

--- Grid Search Finished ---
Best parameters found: {'bidirectional': True, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}
Best validation loss: 2.3531

Grid Search Results:
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}, 'val_loss': 2.41785728931427, 'val_gaze_accuracy': np.float64(0.4294294294294294), 'val_gaze_precision': 0.33477821402830504, 'val_gaze_f1': 0.3661275000411813, 'val_laughter_accuracy': np.float64(0.6861861861861862), 'val_laughter_precision': 0.7052219077535534, 'val_laughter_f1': 0.6934316625471049}
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}, 'val_loss': 2.440830694304572, 'val_gaze_accuracy': np.float64(0.43843843843843844), 'val_gaze_precision': 0.3427165041760301, 'val_gaze_f

,params,val_loss,val_gaze_accuracy,val_gaze_precision,val_gaze_f1,val_laughter_accuracy,val_laughter_precision,val_laughter_f1
0,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.417857,0.429429,0.334778,0.366128,0.686186,0.705222,0.693432
1,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.440831,0.438438,0.342717,0.374082,0.714715,0.726295,0.719428
2,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.401656,0.435435,0.523462,0.376479,0.740240,0.732956,0.735817
3,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.448757,0.445946,0.351448,0.380617,0.656156,0.717504,0.670538
4,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.402290,0.429429,0.334164,0.365368,0.702703,0.714626,0.707614
5,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.456867,0.429429,0.334164,0.365368,0.726727,0.749989,0.734334
6,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.453510,0.435435,0.529094,0.379198,0.696697,0.705887,0.700676
7,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.400112,0.441441,0.345426,0.376825,0.728228,0.749834,0.735461
8,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",2.353098,0.450450,0.349611,0.382942,0.696697,0.705887,0.700676
9,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",2.470768,0.445946,0.351974,0.380774,0.693694,0.715080,0.701454


In [ ]:
# Define dataset parameters
dataset_params = DatasetParams(
    isGazeRelation=False,
    isMixedPhases=True,
    train_size=0.5,
    val_size=0.25,
    n_parts=8
)

# Run the grid search
results4, best_params = grid_search(
    model=EnfantReactionPredictorMultiLSTM,
    dataset_params=dataset_params,
    param_grid=param_grid
)

print("\nGrid Search Results:")
for res in results4:
      print(res)

results_df_4 = pd.DataFrame(results4)

print("\nGrid Search Results DataFrame:")
display(results_df_4)

results_df_4.to_csv('grid_search_multilstm_results_df_4.csv', index=False)

Starting Grid Search with 16 combinations...


Processing params:   0%|          | 0/16 [00:00<?, ?set/s]


--- Running combination 1/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs':…

Validation Loss for this combo: 2.5286

--- Running combination 2/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


...

Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 2.4961

--- Running combination 16/16 ---
Parameters: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 2.4930

--- Grid Search Finished ---
Best parameters found: {'bidirectional': True, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}
Best validation loss: 2.4113

Grid Search Results:
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}, 'val_loss': 2.528639899359809, 'val_gaze_accuracy': np.float64(0.45659928656361476), 'val_gaze_precision': 0.5554873626051534, 'val_gaze_f1': 0.43912663334352303, 'val_laughter_accuracy': np.float64(0.6480380499405469), 'val_laughter_precision': 0.6558396208767225, 'val_laughter_f1': 0.6513360604102968}
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}, 'val_loss': 2.6344079309039645, 'val_gaze_accuracy': np.float64(0.44589774078478), 'val_gaze_precision': 0.5835166319440594, 'val_gaze_f

,params,val_loss,val_gaze_accuracy,val_gaze_precision,val_gaze_f1,val_laughter_accuracy,val_laughter_precision,val_laughter_f1
0,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.528640,0.456599,0.555487,0.439127,0.648038,0.655840,0.651336
1,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.634408,0.445898,0.583517,0.433484,0.686088,0.681256,0.683280
2,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.525762,0.464923,0.567545,0.451389,0.703924,0.706691,0.705187
3,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.687527,0.435196,0.479161,0.431455,0.671819,0.693994,0.678360
4,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.432689,0.467301,0.597382,0.455887,0.659929,0.662762,0.661254
5,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.831067,0.464923,0.588312,0.443265,0.651605,0.655986,0.653593
6,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.412403,0.447087,0.552705,0.430146,0.657551,0.670302,0.662328
7,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.727318,0.466112,0.590606,0.445575,0.669441,0.695467,0.676537
8,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",2.411255,0.461356,0.565709,0.447995,0.621879,0.620035,0.620926
9,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",2.716410,0.466112,0.590606,0.445575,0.652794,0.663689,0.657077


In [ ]:
# Define dataset parameters
dataset_params = DatasetParams(
    isGazeRelation=False,
    isMixedPhases=True,
    train_size=0.5,
    val_size=0.25,
    n_parts=12
)

# Run the grid search
results5, best_params = grid_search(
    model=EnfantReactionPredictorMultiLSTM,
    dataset_params=dataset_params,
    param_grid=param_grid
)

print("\nGrid Search Results:")
for res in results5:
      print(res)

results_df_5 = pd.DataFrame(results5)

print("\nGrid Search Results DataFrame:")
display(results_df_5)

results_df_5.to_csv('grid_search_multilstm_results_df_5.csv', index=False)

Starting Grid Search with 16 combinations...


Processing params:   0%|          | 0/16 [00:00<?, ?set/s]


--- Running combination 1/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs':…

Validation Loss for this combo: 2.1568

--- Running combination 2/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


...

Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 2.2062

--- Running combination 16/16 ---
Parameters: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 2.2806

--- Grid Search Finished ---
Best parameters found: {'bidirectional': True, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}
Best validation loss: 2.0816

Grid Search Results:
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}, 'val_loss': 2.156835322196667, 'val_gaze_accuracy': np.float64(0.4637336504161712), 'val_gaze_precision': 0.3662923131140619, 'val_gaze_f1': 0.3972915719117599, 'val_laughter_accuracy': np.float64(0.7835909631391201), 'val_laughter_precision': 0.7813607748235889, 'val_laughter_f1': 0.7821770383973793}
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}, 'val_loss': 2.142436513533959, 'val_gaze_accuracy': np.float64(0.469678953626635), 'val_gaze_precision': 0.36549821420298745, 'val_gaze_f1

,params,val_loss,val_gaze_accuracy,val_gaze_precision,val_gaze_f1,val_laughter_accuracy,val_laughter_precision,val_laughter_f1
0,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.156835,0.463734,0.366292,0.397292,0.783591,0.781361,0.782177
1,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.142437,0.469679,0.365498,0.399328,0.753864,0.747615,0.745429
2,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.126244,0.473246,0.376743,0.405502,0.803805,0.805637,0.804579
3,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.149758,0.463734,0.367543,0.397794,0.799049,0.801297,0.799976
4,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.212202,0.461356,0.365520,0.395737,0.797860,0.796258,0.796887
5,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.149113,0.463734,0.364272,0.395113,0.771700,0.767057,0.767297
6,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.147483,0.463734,0.366292,0.397292,0.802616,0.804273,0.803327
7,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.287875,0.479191,0.383206,0.410073,0.814507,0.819765,0.816214
8,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",2.081608,0.487515,0.575904,0.433477,0.794293,0.793554,0.793894
9,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",2.175740,0.478002,0.558595,0.422444,0.788347,0.791723,0.789666


### Results - with Gaze Relation

In [ ]:
# Define the parameter grid
param_grid = {
    'hidden_dim': [128, 256],
    'num_layers': [1, 2],
    'dropout': [0.1, 0.3],
    'bidirectional': [False, True],
    'lr': [0.003],
    'weight_decay': [1e-5],
    'num_epochs': [10]
}

In [ ]:
# Define dataset parameters
dataset_params = DatasetParams(
    isGazeRelation=True,
    isMixedPhases=False,
    train_size=0.5,
    val_size=0.25,
    n_parts=1
)

# Run the grid search
results_gr, best_params = grid_search(
    model=EnfantReactionPredictorMultiLSTM,
    dataset_params=dataset_params,
    param_grid=param_grid
)

print("\nGrid Search Results:")
for res in results_gr:
      print(res)

results_gr_df = pd.DataFrame(results_gr)

print("\nGrid Search Results DataFrame:")
display(results_gr_df)

results_gr_df.to_csv('grid_search_multilstm_results_gr_df.csv', index=False)

Starting Grid Search with 16 combinations...


Processing params:   0%|          | 0/16 [00:00<?, ?set/s]


--- Running combination 1/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs':…

Validation Loss for this combo: 3.1458

--- Running combination 2/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


...

Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 2.6794

--- Running combination 16/16 ---
Parameters: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 2.4640

--- Grid Search Finished ---
Best parameters found: {'bidirectional': True, 'dropout': 0.1, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}
Best validation loss: 2.3818

Grid Search Results:
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}, 'val_loss': 3.145810127258301, 'val_gaze_accuracy': np.float64(0.21033653846153846), 'val_gaze_precision': 0.04424145941198225, 'val_gaze_f1': 0.07310604613856848, 'val_laughter_accuracy': np.float64(0.9002403846153846), 'val_laughter_precision': 0.8104327500924556, 'val_laughter_f1': 0.8529791879530968}
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}, 'val_loss': 3.0560524463653564, 'val_gaze_accuracy': np.float64(0.21033653846153846), 'val_gaze_precision': 0.04424145941198225, 'val_g

,params,val_loss,val_gaze_accuracy,val_gaze_precision,val_gaze_f1,val_laughter_accuracy,val_laughter_precision,val_laughter_f1
0,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",3.145810,0.210337,0.044241,0.073106,0.900240,0.810433,0.852979
1,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",3.056052,0.210337,0.044241,0.073106,0.900240,0.810433,0.852979
2,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.650409,0.460337,0.239184,0.312111,0.717548,0.865596,0.770467
3,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.951397,0.210337,0.044241,0.073106,0.099760,0.009952,0.018098
4,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",3.257787,0.210337,0.044241,0.073106,0.824519,0.849691,0.836156
5,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",3.462721,0.210337,0.044241,0.073106,0.900240,0.810433,0.852979
6,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.494491,0.418269,0.214908,0.282363,0.852163,0.857582,0.854807
7,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",3.114915,0.210337,0.044241,0.073106,0.099760,0.009952,0.018098
8,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",3.132920,0.210337,0.044241,0.073106,0.634615,0.868306,0.707809
9,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",3.057257,0.210337,0.044241,0.073106,0.099760,0.009952,0.018098


In [ ]:
# Define dataset parameters
dataset_params = DatasetParams(
    isGazeRelation=True,
    isMixedPhases=True,
    train_size=0.7,
    val_size=0.15,
    n_parts=4
)

# Run the grid search
results_gr2, best_params = grid_search(
    model=EnfantReactionPredictorMultiLSTM,
    dataset_params=dataset_params,
    param_grid=param_grid
)

print("\nGrid Search Results:")
for res in results_gr2:
      print(res)

results_gr_df_2 = pd.DataFrame(results_gr2)

print("\nGrid Search Results DataFrame:")
display(results_gr_df_2)

results_gr_df_2.to_csv('grid_search_multilstm_results_gr_df_2.csv', index=False)

Starting Grid Search with 16 combinations...


Processing params:   0%|          | 0/16 [00:00<?, ?set/s]


--- Running combination 1/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs':…

Validation Loss for this combo: 2.7840

--- Running combination 2/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


...

Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 2.9702

--- Running combination 16/16 ---
Parameters: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 3.0674

--- Grid Search Finished ---
Best parameters found: {'bidirectional': False, 'dropout': 0.3, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}
Best validation loss: 2.6081

Grid Search Results:
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}, 'val_loss': 2.7839668095111847, 'val_gaze_accuracy': np.float64(0.45634920634920634), 'val_gaze_precision': 0.29865785755838115, 'val_gaze_f1': 0.3610357645167772, 'val_laughter_accuracy': np.float64(0.48214285714285715), 'val_laughter_precision': 0.5099521817095038, 'val_laughter_f1': 0.4908938120702826}
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}, 'val_loss': 2.754843771457672, 'val_gaze_accuracy': np.float64(0.4781746031746032), 'val_gaze_precision': 0.3129414942242167, 'val_ga

,params,val_loss,val_gaze_accuracy,val_gaze_precision,val_gaze_f1,val_laughter_accuracy,val_laughter_precision,val_laughter_f1
0,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.783967,0.456349,0.298658,0.361036,0.482143,0.509952,0.490894
1,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.754844,0.478175,0.312941,0.378303,0.436508,0.445996,0.440855
2,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.832826,0.478175,0.294442,0.364462,0.478175,0.498034,0.485661
3,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.814432,0.478175,0.312941,0.378303,0.488095,0.584954,0.477476
4,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.633379,0.478175,0.312941,0.378303,0.511905,0.548342,0.520372
5,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.608106,0.476190,0.318309,0.381563,0.545635,0.587537,0.553208
6,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.766397,0.464286,0.301484,0.365579,0.513889,0.555276,0.521991
7,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",3.044952,0.450397,0.311909,0.368574,0.470238,0.456877,0.462857
8,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",2.712311,0.464286,0.301484,0.365579,0.523810,0.562525,0.531963
9,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",2.978754,0.478175,0.312941,0.378303,0.488095,0.584954,0.477476


In [ ]:
# Define dataset parameters
dataset_params = DatasetParams(
    isGazeRelation=True,
    isMixedPhases=True,
    train_size=0.6,
    val_size=0.2,
    n_parts=8
)

# Run the grid search
results_gr3, best_params = grid_search(
    model=EnfantReactionPredictorMultiLSTM,
    dataset_params=dataset_params,
    param_grid=param_grid
)

print("\nGrid Search Results:")
for res in results_gr3:
      print(res)

results_gr_df_3 = pd.DataFrame(results_gr3)

print("\nGrid Search Results DataFrame:")
display(results_gr_df_3)

results_gr_df_3.to_csv('grid_search_multilstm_results_gr_df_3.csv', index=False)

Starting Grid Search with 16 combinations...


Processing params:   0%|          | 0/16 [00:00<?, ?set/s]


--- Running combination 1/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs':…

Validation Loss for this combo: 2.4037

--- Running combination 2/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


...

Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 2.4690

--- Running combination 16/16 ---
Parameters: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 2.3884

--- Grid Search Finished ---
Best parameters found: {'bidirectional': True, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}
Best validation loss: 2.2893

Grid Search Results:
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}, 'val_loss': 2.4037367436620922, 'val_gaze_accuracy': np.float64(0.45045045045045046), 'val_gaze_precision': 0.3533661210932627, 'val_gaze_f1': 0.385297561401212, 'val_laughter_accuracy': np.float64(0.7012012012012012), 'val_laughter_precision': 0.6965193118944807, 'val_laughter_f1': 0.6986680148242979}
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}, 'val_loss': 2.4787258903185525, 'val_gaze_accuracy': np.float64(0.44294294294294295), 'val_gaze_precision': 0.36597876584504, 'val_gaze_f

,params,val_loss,val_gaze_accuracy,val_gaze_precision,val_gaze_f1,val_laughter_accuracy,val_laughter_precision,val_laughter_f1
0,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.403737,0.450450,0.353366,0.385298,0.701201,0.696519,0.698668
1,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.478726,0.442943,0.365979,0.385354,0.690691,0.710083,0.697975
2,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.374256,0.399399,0.394799,0.377363,0.752252,0.753350,0.752785
3,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.362302,0.429429,0.334164,0.365368,0.714715,0.731183,0.720896
4,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.368141,0.441441,0.344981,0.376819,0.696697,0.693214,0.694855
5,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.471182,0.423423,0.341581,0.369278,0.666667,0.671565,0.668972
6,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.441377,0.448949,0.587631,0.393593,0.740240,0.745412,0.742539
7,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.308682,0.450450,0.354915,0.385688,0.690691,0.713264,0.698793
8,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",2.341334,0.429429,0.373114,0.376262,0.696697,0.702124,0.699188
9,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",2.289318,0.429429,0.334164,0.365368,0.672673,0.700371,0.682292


In [ ]:
# Define dataset parameters
dataset_params = DatasetParams(
    isGazeRelation=True,
    isMixedPhases=True,
    train_size=0.5,
    val_size=0.25,
    n_parts=8
)

# Run the grid search
results_gr4, best_params = grid_search(
    model=EnfantReactionPredictorMultiLSTM,
    dataset_params=dataset_params,
    param_grid=param_grid
)

print("\nGrid Search Results:")
for res in results_gr4:
      print(res)

results_gr_df_4 = pd.DataFrame(results_gr4)

print("\nGrid Search Results DataFrame:")
display(results_gr_df_4)

results_gr_df_4.to_csv('grid_search_multilstm_results_gr_df_4.csv', index=False)

Starting Grid Search with 16 combinations...


Processing params:   0%|          | 0/16 [00:00<?, ?set/s]


--- Running combination 1/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs':…

Validation Loss for this combo: 2.3936

--- Running combination 2/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


...

Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 2.4248

--- Running combination 16/16 ---
Parameters: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 2.5563

--- Grid Search Finished ---
Best parameters found: {'bidirectional': False, 'dropout': 0.3, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}
Best validation loss: 2.2788

Grid Search Results:
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}, 'val_loss': 2.39358381430308, 'val_gaze_accuracy': np.float64(0.4756242568370987), 'val_gaze_precision': 0.6015422152813314, 'val_gaze_f1': 0.46457948472436994, 'val_laughter_accuracy': np.float64(0.6860879904875149), 'val_laughter_precision': 0.6812564723670466, 'val_laughter_f1': 0.6832802086351781}
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}, 'val_loss': 2.66776090198093, 'val_gaze_accuracy': np.float64(0.4161712247324614), 'val_gaze_precision': 0.5188384306832385, 'val_gaze_f1

,params,val_loss,val_gaze_accuracy,val_gaze_precision,val_gaze_f1,val_laughter_accuracy,val_laughter_precision,val_laughter_f1
0,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.393584,0.475624,0.601542,0.464579,0.686088,0.681256,0.683280
1,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.667761,0.416171,0.518838,0.382375,0.662307,0.661213,0.661745
2,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.450275,0.455410,0.518165,0.462538,0.643282,0.651795,0.646846
3,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.743375,0.441141,0.494822,0.441793,0.695600,0.688481,0.691008
4,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.278782,0.410226,0.483212,0.415532,0.693222,0.687247,0.689559
5,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.429117,0.412604,0.547149,0.416158,0.678954,0.675417,0.676997
6,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.417159,0.450654,0.540643,0.427898,0.695600,0.688481,0.691008
7,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.612203,0.436385,0.496854,0.440235,0.695600,0.688481,0.691008
8,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",2.389067,0.460166,0.582551,0.439997,0.712247,0.704069,0.706298
9,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",2.530114,0.437574,0.493723,0.440704,0.709869,0.701263,0.703531


In [ ]:
# Define dataset parameters
dataset_params = DatasetParams(
    isGazeRelation=True,
    isMixedPhases=True,
    train_size=0.5,
    val_size=0.25,
    n_parts=12
)

# Run the grid search
results_gr5, best_params = grid_search(
    model=EnfantReactionPredictorMultiLSTM,
    dataset_params=dataset_params,
    param_grid=param_grid
)

print("\nGrid Search Results:")
for res in results_gr5:
      print(res)

results_gr_df_5 = pd.DataFrame(results_gr5)

print("\nGrid Search Results DataFrame:")
display(results_gr_df_5)

results_gr_df_5.to_csv('grid_search_multilstm_results_gr_df_5.csv', index=False)


Starting Grid Search with 16 combinations...


Processing params:   0%|          | 0/16 [00:00<?, ?set/s]


--- Running combination 1/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs':…

Validation Loss for this combo: 2.1287

--- Running combination 2/16 ---
Parameters: {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


...

Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 2.1468

--- Running combination 16/16 ---
Parameters: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}


Training Epochs (Params {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 256, 'lr': 0.003, 'num_epochs': …

Validation Loss for this combo: 2.1551

--- Grid Search Finished ---
Best parameters found: {'bidirectional': True, 'dropout': 0.3, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}
Best validation loss: 2.0842

Grid Search Results:
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 1, 'weight_decay': 1e-05}, 'val_loss': 2.1286790737738976, 'val_gaze_accuracy': np.float64(0.44589774078478), 'val_gaze_precision': 0.23429922103581724, 'val_gaze_f1': 0.2915746141021514, 'val_laughter_accuracy': np.float64(0.8145065398335315), 'val_laughter_precision': 0.8119790089186875, 'val_laughter_f1': 0.8122656745189809}
{'params': {'bidirectional': False, 'dropout': 0.1, 'hidden_dim': 128, 'lr': 0.003, 'num_epochs': 10, 'num_layers': 2, 'weight_decay': 1e-05}, 'val_loss': 2.1497595493610087, 'val_gaze_accuracy': np.float64(0.469678953626635), 'val_gaze_precision': 0.36549821420298745, 'val_gaze_f

,params,val_loss,val_gaze_accuracy,val_gaze_precision,val_gaze_f1,val_laughter_accuracy,val_laughter_precision,val_laughter_f1
0,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.128679,0.445898,0.234299,0.291575,0.814507,0.811979,0.812266
1,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.149760,0.469679,0.365498,0.399328,0.731272,0.729717,0.730427
2,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.131462,0.448276,0.234476,0.292825,0.816885,0.819544,0.817909
3,"{'bidirectional': False, 'dropout': 0.1, 'hidd...",2.220650,0.470868,0.374678,0.403430,0.806183,0.813313,0.808331
4,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.162762,0.448276,0.234899,0.292618,0.788347,0.787442,0.787852
5,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.166387,0.469679,0.365498,0.399328,0.775268,0.775099,0.775182
6,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.148116,0.463734,0.366292,0.397292,0.790725,0.789558,0.790067
7,"{'bidirectional': False, 'dropout': 0.3, 'hidd...",2.112359,0.463734,0.363504,0.395422,0.781213,0.781899,0.781538
8,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",2.106797,0.445898,0.233876,0.291782,0.789536,0.788778,0.789128
9,"{'bidirectional': True, 'dropout': 0.1, 'hidde...",2.164789,0.461356,0.357247,0.392399,0.760999,0.755803,0.756258


## Training Best Model

In [ ]:
dataset_params = DatasetParams(
    isGazeRelation=True,
    isMixedPhases=False,
    train_size=0.5,
    val_size=0.25,
    n_parts=1
)

model_params = ModelParams(
    hidden_dim=256,
    bidirectional=False,
    num_layers=1,
    dropout=0.3,
    input_size=416
)

train_params = TrainParams(
    lr = 0.003,
    weight_decay = 1e-5,
    num_epochs = 25
)

model = training_model(EnfantReactionPredictorMultiLSTM, dataset_params, model_params, train_params)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Starting training...


Training Epochs:   0%|          | 0/25 [00:00<?, ?it/s]

Epoch [1/25], 	Train Loss: 3.7287, Train Gaze Acc: 0.6651, Train Gaze Prec: 0.4423, Train Gaze F1: 0.5313, Train Laughter Acc: 0.6206, Train Laughter Prec: 0.3851, Train Laughter F1: 0.4753, 
	Val Loss: 3.7988, Val Gaze Acc: 0.5187, Val Gaze Prec: 0.2690, Val Gaze F1: 0.3543, Val Laughter Acc: 0.4693, Val Laughter Prec: 0.2203, Val Laughter F1: 0.2998
Epoch [2/25], 	Train Loss: 3.2183, Train Gaze Acc: 0.6651, Train Gaze Prec: 0.4423, Train Gaze F1: 0.5313, Train Laughter Acc: 0.6206, Train Laughter Prec: 0.3851, Train Laughter F1: 0.4753, 
	Val Loss: 3.3978, Val Gaze Acc: 0.5187, Val Gaze Prec: 0.2690, Val Gaze F1: 0.3543, Val Laughter Acc: 0.4693, Val Laughter Prec: 0.2203, Val Laughter F1: 0.2998
Epoch [3/25], 	Train Loss: 2.5748, Train Gaze Acc: 0.6651, Train Gaze Prec: 0.4423, Train Gaze F1: 0.5313, Train Laughter Acc: 0.6206, Train Laughter Prec: 0.3851, Train Laughter F1: 0.4753, 
	Val Loss: 2.9348, Val Gaze Acc: 0.5187, Val Gaze Prec: 0.2690, Val Gaze F1: 0.3543, Val Laughter Ac

In [ ]:
dataset_params = DatasetParams(
    isGazeRelation=True,
    isMixedPhases=True,
    train_size=0.5,
    val_size=0.25,
    n_parts=12
)

model_params = ModelParams(
    hidden_dim=256,
    bidirectional=True,
    num_layers=2,
    dropout=0.1,
    input_size=416
)

train_params = TrainParams(
    lr = 0.003,
    weight_decay = 1e-5,
    num_epochs = 25
)

model2 = training_model(EnfantReactionPredictorMultiLSTM, dataset_params, model_params, train_params)

Starting training...


Training Epochs:   0%|          | 0/25 [00:00<?, ?it/s]

Epoch [1/25], 	Train Loss: 2.0509, Train Gaze Acc: 0.5685, Train Gaze Prec: 0.5818, Train Gaze F1: 0.5373, Train Laughter Acc: 0.7037, Train Laughter Prec: 0.7285, Train Laughter F1: 0.7109, 
	Val Loss: 2.2054, Val Gaze Acc: 0.5374, Val Gaze Prec: 0.4426, Val Gaze F1: 0.4608, Val Laughter Acc: 0.6703, Val Laughter Prec: 0.6796, Val Laughter F1: 0.6733
Epoch [2/25], 	Train Loss: 1.4285, Train Gaze Acc: 0.6397, Train Gaze Prec: 0.6516, Train Gaze F1: 0.6176, Train Laughter Acc: 0.7079, Train Laughter Prec: 0.7361, Train Laughter F1: 0.7156, 
	Val Loss: 1.9974, Val Gaze Acc: 0.5785, Val Gaze Prec: 0.5315, Val Gaze F1: 0.5040, Val Laughter Acc: 0.6715, Val Laughter Prec: 0.6880, Val Laughter F1: 0.6755
Epoch [3/25], 	Train Loss: 1.4132, Train Gaze Acc: 0.5932, Train Gaze Prec: 0.5692, Train Gaze F1: 0.5518, Train Laughter Acc: 0.7127, Train Laughter Prec: 0.7346, Train Laughter F1: 0.7194, 
	Val Loss: 1.8837, Val Gaze Acc: 0.5531, Val Gaze Prec: 0.4433, Val Gaze F1: 0.4709, Val Laughter Ac

### 10-fold CV

In [ ]:
def get_targets(y):
    target_laughter_indices = []
    target_gaze_indices = []
    target_gazerelation_indices = []

    for target_dict in y:
        laughter_val = target_dict.get('Laughter@CHI')
        gaze_val = target_dict.get('Gaze@CHI')
        gazerelation_val = target_dict.get('GazeRelation')
        # Use lookup for converting values to indices
        target_laughter_indices.append(lookup(vocab_laughter_chi, laughter_val))
        target_gaze_indices.append(lookup(vocab_gaze_chi, gaze_val))
        target_gazerelation_indices.append(lookup(vocab_gazerelation, gazerelation_val))

    targets_laughter = torch.tensor(target_laughter_indices, dtype=torch.long)
    targets_gaze = torch.tensor(target_gaze_indices, dtype=torch.long)
    targets_gazerelation = torch.tensor(target_gazerelation_indices, dtype=torch.long)

    return targets_gaze, targets_laughter, targets_gazerelation

In [ ]:
FEATURE_KEYS = ["Gaze@MOT", "Utterance@MOT", "Prosody@MOT", "Facial@MOT", "Laughter@MOT", "Action@MOT"]

def to_sequence_of_dicts(sample):
    """
    Convert a sample to List[Dict] (sequence of step dicts).
    Accepts:
      - List[Dict] (already good)
      - Dict with list-valued fields (time-major) -> sequence of dicts
      - Dict with scalar fields -> single-step sequence
    """
    # Case 1: sample is already a sequence of step dicts
    if isinstance(sample, list):
        if len(sample) == 0:
            return []  # empty sequence (skip later if needed)
        if isinstance(sample[0], dict):
            return sample
        raise TypeError(f"List sample elements must be dicts, got {type(sample[0])}")

    # Case 2: sample is a dict (either time-major or single-step)
    if isinstance(sample, dict):
        # detect time-major: values that are lists with equal-ish lengths
        lengths = []
        for k in FEATURE_KEYS:
            v = sample.get(k, None)
            if isinstance(v, list):
                lengths.append(len(v))
        if lengths:
            T = min(lengths)  # be safe if lengths slightly mismatch
            seq = []
            for t in range(T):
                step = {}
                for k in FEATURE_KEYS:
                    v = sample.get(k, None)
                    if isinstance(v, list):
                        step[k] = v[t]
                    else:
                        step[k] = v  # broadcast scalar
                seq.append(step)
            return seq
        else:
            # treat as a single-step record
            return [sample]

    # Anything else is invalid
    raise TypeError(f"Unsupported sample type: {type(sample)}")

def make_batches(X, batch_size):
    n = len(X)
    for start in range(0, n, batch_size):
        end = min(start + batch_size, n)
        # Normalize each sample into a sequence of dicts
        batch = [to_sequence_of_dicts(x) for x in X[start:end]]
        # Optionally drop empty sequences
        batch = [seq for seq in batch if len(seq) > 0]
        if len(batch) == 0:
            continue
        yield batch, start, end  # keep indices to slice labels


In [ ]:
from sklearn.model_selection import KFold

def CrossValidation(dataset_params: DatasetParams, model_params: ModelParams, train_params: TrainParams):
    # Combine train and test data for cross-validation
    X_combined, vocabs = get_data(dataset_params.isGazeRelation)
    # Initialize KFold
    kf = KFold(n_splits=10, shuffle=False)

    test_losses = []
    test_gaze_accuracies = []
    test_laughter_accuracies = []

    print("Starting 10-Fold Cross-Validation...")

    # Iterate over each fold
    for fold, (train_index, test_index) in tqdm(enumerate(kf.split(X_combined)), total=kf.get_n_splits(), desc="Cross-Validation Progress"):
        print(f"\n--- Fold {fold+1}/10 ---")

        # Split data into train and test sets for the current fold
        train_fold = [X_combined[i] for i in train_index]
        test_fold = [X_combined[i] for i in test_index]

        X_train_fold, y_train_fold = DatasetSplit(dataset = X, n_parts = 1, train_size = 0, val_size = 0).get_features_and_targets(train_fold)
        X_test_fold, y_test_fold = DatasetSplit(dataset = X, n_parts = 1, train_size = 0, val_size = 0).get_features_and_targets(test_fold)

        y_train_fold_gaze, y_train_fold_laughter, y_train_fold_gazerelation = get_targets(y_train_fold)
        y_test_fold_gaze, y_test_fold_laughter, y_test_fold_gazerelation = get_targets(y_test_fold)

        y_train_fold_gaze = torch.as_tensor(y_train_fold_gaze, dtype=torch.long, device=device)
        y_train_fold_laughter = torch.as_tensor(y_train_fold_laughter, dtype=torch.long, device=device)
        if dataset_params.isGazeRelation:
            y_train_fold_gazerelation = torch.as_tensor(y_train_fold_gazerelation, dtype=torch.long, device=device)

        y_test_fold_gaze = torch.as_tensor(y_test_fold_gaze, dtype=torch.long, device=device)
        y_test_fold_laughter = torch.as_tensor(y_test_fold_laughter, dtype=torch.long, device=device)
        if dataset_params.isGazeRelation:
            y_test_fold_gazerelation = torch.as_tensor(y_test_fold_gazerelation, dtype=torch.long, device=device)


        # Initialize a new model for each fold to ensure independent training
        model = EnfantReactionPredictorMultiLSTM(
            vocabs,
            dataset_params.isGazeRelation,
            model_params.hidden_dim,
            model_params.num_layers,
            model_params.dropout,
            model_params.input_size,
            model_params.bidirectional
        ).to(device)

        optimizer = optim.Adam(model.parameters(), lr=0.003, weight_decay=1e-5)
        criterion = nn.CrossEntropyLoss()

        num_epochs_fold = 10
        for epoch in range(num_epochs_fold):
            model.train()
            X_train_norm = [to_sequence_of_dicts(x) for x in X_train_fold]

            (logits_gaze, logits_laughter, logits_gazerelation) = model(X_train_norm, device)

            loss_gaze = criterion(logits_gaze.squeeze(0), y_train_fold_gaze)
            loss_laughter = criterion(logits_laughter.squeeze(0), y_train_fold_laughter)
            loss_gazerelation = criterion(logits_gazerelation.squeeze(0), y_train_fold_gazerelation)
            loss = loss_gaze + loss_laughter + loss_gazerelation

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Evaluate on the test set for the current fold
        model.eval()
        with torch.no_grad():
            X_test_norm = [to_sequence_of_dicts(x) for x in X_test_fold]
            (test_logits_gaze, test_logits_laughter, test_logits_gazerelation) = model(X_test_norm, device)

            test_loss_gaze = criterion(test_logits_gaze.squeeze(0), y_test_fold_gaze)
            test_loss_laughter = criterion(test_logits_laughter.squeeze(0), y_test_fold_laughter)
            test_loss_gazerelation = criterion(test_logits_gazerelation.squeeze(0), y_test_fold_gazerelation)
            test_total_loss = test_loss_gaze + test_loss_laughter + test_loss_gazerelation

            test_pred_gaze = torch.argmax(test_logits_gaze.squeeze(0), dim=1)
            test_correct_gaze = (test_pred_gaze == y_test_fold_gaze).sum().item()
            test_accuracy_gaze = test_correct_gaze / y_test_fold_gaze.size(0)

            test_pred_laughter = torch.argmax(test_logits_laughter.squeeze(0), dim=1)
            test_correct_laughter = (test_pred_laughter == y_test_fold_laughter).sum().item()
            test_accuracy_laughter = test_correct_laughter / y_test_fold_laughter.size(0)


        test_losses.append(test_total_loss.item())
        test_gaze_accuracies.append(test_accuracy_gaze)
        test_laughter_accuracies.append(test_accuracy_laughter)

        print(f"Fold {fold+1} - Test Loss: {test_total_loss:.4f}, Test Gaze Acc: {test_accuracy_gaze:.4f}, Test Laughter Acc: {test_accuracy_laughter:.4f}")

    # Report average performance across all folds
    print("\n--- Cross-Validation Results ---")
    print(f"Average Test Loss: {np.mean(test_losses):.4f}")
    print(f"Average Test Gaze Accuracy: {np.mean(test_gaze_accuracies):.4f}")
    print(f"Average Test Laughter Accuracy: {np.mean(test_laughter_accuracies):.4f}")

In [ ]:
dataset_params = DatasetParams(
    isGazeRelation=True,
    isMixedPhases=False,
    train_size=0.5,
    val_size=0.25,
    n_parts=1
)

model_params = ModelParams(
    hidden_dim=256,
    bidirectional=False,
    num_layers=1,
    dropout=0.3,
    input_size=416
)

train_params = TrainParams(
    lr = 0.003,
    weight_decay = 1e-5,
    num_epochs = 25
)

CrossValidation(dataset_params, model_params, train_params)

Starting 10-Fold Cross-Validation...


Cross-Validation Progress:   0%|          | 0/10 [00:00<?, ?it/s]


--- Fold 1/10 ---
Fold 1 - Test Loss: 3.1571, Test Gaze Acc: 0.6036, Test Laughter Acc: 0.4685

--- Fold 2/10 ---
Fold 2 - Test Loss: 2.4131, Test Gaze Acc: 0.8318, Test Laughter Acc: 0.7748

--- Fold 3/10 ---
Fold 3 - Test Loss: 2.2168, Test Gaze Acc: 0.6066, Test Laughter Acc: 0.6246

--- Fold 4/10 ---
Fold 4 - Test Loss: 4.7621, Test Gaze Acc: 0.0300, Test Laughter Acc: 0.8769

--- Fold 5/10 ---
Fold 5 - Test Loss: 3.0253, Test Gaze Acc: 0.5135, Test Laughter Acc: 0.7207

--- Fold 6/10 ---
Fold 6 - Test Loss: 2.9084, Test Gaze Acc: 0.6186, Test Laughter Acc: 0.6156

--- Fold 7/10 ---
Fold 7 - Test Loss: 3.3374, Test Gaze Acc: 0.4789, Test Laughter Acc: 0.7530

--- Fold 8/10 ---
Fold 8 - Test Loss: 2.5440, Test Gaze Acc: 0.6627, Test Laughter Acc: 0.7861

--- Fold 9/10 ---
Fold 9 - Test Loss: 2.8768, Test Gaze Acc: 0.4217, Test Laughter Acc: 0.8223

--- Fold 10/10 ---
Fold 10 - Test Loss: 3.9766, Test Gaze Acc: 0.3675, Test Laughter Acc: 0.6024

--- Cross-Validation Results ---
Aver

In [ ]:
dataset_params = DatasetParams(
    isGazeRelation=True,
    isMixedPhases=True,
    train_size=0.5,
    val_size=0.25,
    n_parts=12
)

model_params = ModelParams(
    hidden_dim=256,
    bidirectional=True,
    num_layers=2,
    dropout=0.1,
    input_size=416
)

train_params = TrainParams(
    lr = 0.003,
    weight_decay = 1e-5,
    num_epochs = 25
)

CrossValidation(dataset_params, model_params, train_params)

Starting 10-Fold Cross-Validation...


Cross-Validation Progress:   0%|          | 0/10 [00:00<?, ?it/s]


--- Fold 1/10 ---
Fold 1 - Test Loss: 2.9479, Test Gaze Acc: 0.5976, Test Laughter Acc: 0.5375

--- Fold 2/10 ---
Fold 2 - Test Loss: 2.3093, Test Gaze Acc: 0.8258, Test Laughter Acc: 0.7147

--- Fold 3/10 ---
Fold 3 - Test Loss: 1.9699, Test Gaze Acc: 0.8168, Test Laughter Acc: 0.6246

--- Fold 4/10 ---
Fold 4 - Test Loss: 4.8289, Test Gaze Acc: 0.0300, Test Laughter Acc: 0.8769

--- Fold 5/10 ---
Fold 5 - Test Loss: 2.6132, Test Gaze Acc: 0.6577, Test Laughter Acc: 0.7778

--- Fold 6/10 ---
Fold 6 - Test Loss: 3.4385, Test Gaze Acc: 0.5646, Test Laughter Acc: 0.6336

--- Fold 7/10 ---
Fold 7 - Test Loss: 3.1202, Test Gaze Acc: 0.4729, Test Laughter Acc: 0.8072

--- Fold 8/10 ---
Fold 8 - Test Loss: 2.5725, Test Gaze Acc: 0.6265, Test Laughter Acc: 0.8072

--- Fold 9/10 ---
Fold 9 - Test Loss: 2.7200, Test Gaze Acc: 0.4217, Test Laughter Acc: 0.8193

--- Fold 10/10 ---
Fold 10 - Test Loss: 5.2012, Test Gaze Acc: 0.2500, Test Laughter Acc: 0.5512

--- Cross-Validation Results ---
Aver